In [1]:
#!pip install langdetect

#Reading Data
import nltk
from nltk.probability import FreqDist
import pandas as pd
import datetime as dt
from datetime import datetime as dt
import numpy as np
from collections import OrderedDict
pd.options.mode.chained_assignment = None
import unicodedata
import regex as re
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import pyodbc


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence 

## Enviarselo a Oscar Garavito  

In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server= TPCCP-DB09\SCNEAR;'
                      'Database=GoDaddy;'
                      'Trusted_Connection=yes;')


data ='''SELECT A.[Date]
      ,A.[Time]
      ,A.[ShopperId]
      ,A.[CustomerType]
      ,A.[Guide]
      ,A.[Supervisor]
      ,A.[ShopperRegionI]
      ,A.[SurveyId]
      ,A.[Orecxid]
      ,A.[CaseProductGroup]
      ,A.[CssScore]
      ,A.[NpsScore]
      ,CASE 
		WHEN A.[NpsScore] <=6 THEN 'Detractor' 
		WHEN A.[NpsScore] <=8 THEN 'Passive'
		WHEN A.[NpsScore] <=10 THEN 'Promoter'
		ELSE NULL END AS 'NPS'
	   ,CONCAT(A.Promoter,A.Detractor,A.Passive) as 'Feedback'
	   ,C.SiteCode
	   ,C.Jomax
  FROM [GoDaddy].[dbo].[tbNpsCalls] A INNER JOIN [GoDaddy].[dbo].[tbAgentsId] B ON A.[GuideName]=B.[Name]
	INNER JOIN [GoDaddy].[dbo].[tbHCHistorical] C ON B.Jomax=C.[Jomax] 
	WHERE C.[FechaHC]=(SELECT MAX([FechaHC]) FROM [GoDaddy].[dbo].[tbHCHistorical])
	ORDER BY [Date] DESC'''
BDInfinite = pd.read_sql(sql=data,con=conn)

In [3]:
df=BDInfinite
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax
0,2023-06-11,03:17:59,205148772,None,Angela DeLa Torre,"Saenz, Mayra",International - Top Markets,201592478,107518452.0,account,10,8,Passive,\r,Perú - Lima,angela0040
1,2023-06-11,03:39:37,575907681,None,Jared Basauri,"Saenz, Mayra",International - RoW,201592483,107518513.0,None,10,7,Passive,\r,Perú - Lima,jared5798
2,2023-06-11,04:04:13,68929869,None,Jared Basauri,"Saenz, Mayra",International - Top Markets,201592490,107518579.0,None,10,10,Promoter,\r,Perú - Lima,jared5798
3,2023-06-11,04:18:12,282348646,None,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,Happy with product - been trying to pay online...,Perú - Lima,roberto3014
4,2023-06-11,04:56:19,456716782,None,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,She's very smart and help me out with the issu...,Perú - Lima,estefani5632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17069,2022-09-18,07:43:33,458124180,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201129671,96914746.0,None,10,10,Promoter,\r,Egipt - Cairo,abdelrahman0673
17070,2022-09-18,23:45:37,9633670,Partner,Yusuf Asik,"Caglayan, Gokberk",International - RoW,201130346,96926343.0,hosting,10,10,Promoter,"Müsteri temsicisi Yusuf bey çok ilgiliydi, ken...",Turkey - Istanbul,yusuf9095
17071,2022-09-18,23:51:36,305861822,International Independent,Giovany ChancafePretell,"Cotrina Gomez, Aron",International - RoW,201130360,96926543.0,sitebuilder,9,9,Promoter,very helpful\r,Perú - Lima,giovany1051
17072,2022-09-18,23:44:33,157664005,International Independent,Ahmet Mengir,"Basturk, Emin",International - RoW,201130362,96926314.0,None,9,9,Promoter,\r,Turkey - Istanbul,ahmet0030


In [4]:
df[df['Orecxid']==103805958.0]

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax


In [5]:
df["Feedback"]=df["Feedback"].str.replace('\r',"")
df.to_csv("../Raw Datas/LanguageRaw.csv",encoding="utf-8-sig")

In [6]:
#df=df.dropna()
df["len"]=df["Feedback"].apply(lambda x: len(str(x)))
df=df[df["len"] >4]
#df["Feedback"]=df["Feedback"].str.replace('\r',"")
#df=df.drop("Unnamed: 17", axis=1)
#df=df.drop("Promoter", axis=1)
#df=df.drop("Passive", axis=1)
#df=df.drop("Detractor", axis=1)#Aux_Remove

In [7]:
df.head()

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax,len
3,2023-06-11,04:18:12,282348646,None,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,Happy with product - been trying to pay online...,Perú - Lima,roberto3014,78
4,2023-06-11,04:56:19,456716782,None,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,She's very smart and help me out with the issu...,Perú - Lima,estefani5632,68
5,2023-06-11,03:07:35,81164049,None,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,Bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287
7,2023-06-11,05:38:27,67910895,None,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,Karim was super helpful and very professional,Egipt - Cairo,karim5737,45
8,2023-06-11,06:24:59,137315074,None,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,"Professional, Patient, and very helpful. Thank...",Egipt - Cairo,ahmed8164,81


In [8]:
df.CustomerType.value_counts()

International Independent    7673
Partner                       847
US Independent                492
Investor                      343
Name: CustomerType, dtype: int64

In [9]:
df.to_csv("../Raw Datas/RAWDATASQL.csv",encoding="utf-8-sig")

In [10]:
#LEER DATA
df=pd.read_csv("../Raw Datas/RAWDATASQL.csv")

#############################PREPARACION DE DF ###############################################################
#df=df.drop(["IR"], axis =1)
df['Feedback'] = df['Feedback'].map(str)

#OBTENER PROMOTER DETRACTOR
#def get_promoter(kpi):
#    if (kpi == 100):
#        return "Promoter"
#    elif (kpi == 0):
#        return "Detractor"
#    return "None"

#df["Csat"]=df["C-Sat"].apply(lambda x: get_promoter(x))

#OBTENER EL MES
df["Date"]=pd.to_datetime(df['Date'])
df["Month"]=df.Date.dt.month_name()
df["Year"]=df.Date.dt.year
df=df.drop("Date", axis=1)
df=df.drop("Jomax", axis=1)

df["Feedback"] = df["Feedback"].apply(lambda x: str(x).lower())

#FUNCIOARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df['Feedback'] = df['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))

In [11]:
def get_language(text):
    try:
        language = detect(text)
    except:
        language = None
    return language

#def get_language(text):
#    if (text == 'TP Bogota - NA'):
#        return "en"
#    elif (text == 'TP Lima'):
#        return "en"
#    elif (text == 'TP Bogota - ES'):
#        return "es"
#    elif (text == 'TP Bogota - BR'):
#        return "pt"
#    return "None"

df["Language"] = df["Feedback"].apply(lambda x: get_language(x))

In [12]:
df.head()

,Unnamed: 0,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,len,Month,Year,Language
0,3,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,78,June,2023,en
1,4,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,68,June,2023,en
2,5,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,287,June,2023,en
3,7,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,45,June,2023,en
4,8,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,"professional, patient, and very helpful. thank...",Egipt - Cairo,81,June,2023,en


In [13]:
#LEER DATA ARABIC
df2=pd.read_csv("../Raw Datas/RAWDATASQL.csv")
df2=df2[df2.SiteCode == 'Egipt - Cairo']

#############################PREPARACION DE DF ###############################################################
#df=df.drop(["IR"], axis =1)
df2['Feedback'] = df2['Feedback'].map(str)

#OBTENER EL MES
df2["Date"]=pd.to_datetime(df2['Date'])
df2["Month"]=df2.Date.dt.month_name()
df2["Year"]=df2.Date.dt.year
df2=df2.drop("Date", axis=1)
df2=df2.drop("Jomax", axis=1)

df2["Feedback"] = df2["Feedback"].apply(lambda x: str(x).lower())

def get_language(text):
    try:
        language = detect(text)
    except:
        language = None
    return language

df2["Language"] = df2["Feedback"].apply(lambda x: get_language(x))

In [14]:
df=pd.concat([df, df2])
#df=df.drop_duplicates(subset=['SurveyId'])
df

,Unnamed: 0,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,len,Month,Year,Language
0,3,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,78,June,2023,en
1,4,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,68,June,2023,en
2,5,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,287,June,2023,en
3,7,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,45,June,2023,en
4,8,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,"professional, patient, and very helpful. thank...",Egipt - Cairo,81,June,2023,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,16834,03:13:50,313819021,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201144649,97282115.0,hosting,10,10,Promoter,comment archived,Egipt - Cairo,16,September,2022,en
9511,16892,02:26:26,235418468,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141422,97204539.0,email,10,9,Promoter,comment archived,Egipt - Cairo,16,September,2022,en
9512,16893,08:05:32,529768147,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141634,97206659.0,NaN,2,1,Detractor,"""no support but you say """" its normal"""" that s...",Egipt - Cairo,88,September,2022,en
9580,16999,05:33:26,542100514,International Independent,Arsany Mamdouh,"Ashraf, Youstina",International - RoW,201132730,96992773.0,account,0,0,Detractor,comment archived,Egipt - Cairo,16,September,2022,en


In [15]:
#df=df.drop("index", axis=1)
df=df.drop("Unnamed: 0", axis=1)

##########Raw Data por Idioma ###############################
df.to_csv("../Raw Datas/Coment_con_idioma.csv",encoding="utf-8-sig")

In [16]:
df.Language.value_counts()

en    6884
tr     873
es     654
fr     353
de     286
it     150
af      91
ro      62
ca      59
pt      52
no      51
nl      49
da      47
ar      32
id      30
hr      27
cy      21
et      18
sv      18
so      17
sw      13
sl      12
tl      12
cs       8
fi       7
pl       6
hu       5
lv       5
sk       4
sq       2
vi       1
lt       1
Name: Language, dtype: int64

In [17]:
######################## PROCESAMIENTO EN INGLES ###############################################################
df=pd.read_csv("../Raw Datas/Coment_con_idioma.csv")

df_en=df[df["Language"]=="en"]
df_en["ShopperRegionI"]=df_en["ShopperRegionI"].map(str)
df_en["CaseProductGroup"]=df_en["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_en['index_2'] = df_en[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_en = df_en.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)

#Crea un index doble, por semana y CSAT/Lob. 
df_en = df_en.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_en = df_en.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_en=df_en.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

In [18]:
df_en.head()

Feedback  \
Year Month    index_2                                                                           
2022 December Detractor/Perú - Lima/nan     spent nearly all day on the chat then eventual...   
              Promoter/Perú - Lima/account  comment archived wonderful man with the best m...   
              Promoter/Perú - Lima/hosting  comment archived comment archived great custom...   
              Promoter/Perú - Lima/email    polite and listens ale was fantastic and my ca...   
              Promoter/Perú - Lima/domains  was nice and comfortable victor  was extremely...   

                                                                                          len  
Year Month    index_2                                                                          
2022 December Detractor/Perú - Lima/nan     294 124 95 207 71 47 16 188 115 28 83 16 96 71...  
              Promoter/Perú - Lima/account  16 35 69 33 16 71 16 45 56 33 48 13 46 64 21 1...  
              Promoter/Perú - Lima/hosting           16 16 37 72 32 102 155 16 16 16 20 12 46  
              Promoter/Perú - Lima/email    18 40 12 16 42 13 16 32 13 29 140 243 102 126 ...  
              Promoter/Perú - Lima/domains  24 56 62 71 109 37 17 18 58 115 25 22 16 22 16...

In [19]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_en["Feedback"] = df_en["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_en['Feedback'] = df_en['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_en["words"] = df_en["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

common_words = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei",
                "we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", 
                "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was",
                "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but",
                "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", 
                "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", 
                "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡",
                "there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't",
                "!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish"))))))) #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and w not in common_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_en["words"] = df_en["words"].apply(lambda x: remove_stopwords(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_en["lemma"]=df_en["words"].apply(lambda x: lemmatizacion(x))

df_en=df_en.drop(["words", "Feedback"], axis=1)
df_en.to_csv("../Raw Datas/Feedback_Lematized_en.csv")

###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_en["frecwords"]=df_en["lemma"].apply(lambda x: common_words(x))
df_en=df_en.drop(["lemma"], axis = 1)
df_en.to_csv("../Raw Datas/freqwords.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_english.csv")

In [20]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Detractor,Perú - Lima,nan,website,4
2022,December,Detractor,Perú - Lima,nan,call,4
2022,December,Detractor,Perú - Lima,nan,support,4
2022,December,Detractor,Perú - Lima,nan,godaddy,4
2022,December,Detractor,Perú - Lima,nan,speak,3


In [21]:
########################## PROCESAMIENTO EN ESPAÑOL ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_es=df[df["Language"]=="es"]
df_es["ShopperRegionI"]=df_es["ShopperRegionI"].map(str)
df_es["CaseProductGroup"]=df_es["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_es['index_2'] = df_es[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_es = df_es.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_es = df_es.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_es = df_es.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_es=df_es.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_es.head()

Feedback  \
Year Month    index_2                                                                               
2022 December Promoter/Perú - Lima/nan          estoy esperando propuesta de hector para renov...   
              Promoter/Perú - Lima/hosting      excelente atencion al cliente llam las 2 am me...   
              Promoter/Perú - Lima/sitebuilder  respuesta rpida en espaol, genuina preocupacin...   
              Promoter/Perú - Lima/domains      el mejor servicio al cliente que he podido rec...   
              Detractor/Perú - Lima/email       llevo con problemas en el email desde el 14/12...   

                                                                                              len  
Year Month    index_2                                                                              
2022 December Promoter/Perú - Lima/nan                                      52 60 146 40 28 47 17  
              Promoter/Perú - Lima/hosting                     29 56 35 50 36 202 133 48 64 17 68  
              Promoter/Perú - Lima/sitebuilder                                 61 40 114 95 31 22  
              Promoter/Perú - Lima/domains      100 46 91 109 136 172 68 54 44 31 13 13 142 45...  
              Detractor/Perú - Lima/email                                                  51 145

In [22]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_es["Feedback"] = df_es["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_es['Feedback'] = df_es['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_es["words"] = df_es["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_es = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_es(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_es["words"] = df_es["words"].apply(lambda x: remove_stopwords_es(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_es["lemma"]=df_es["words"].apply(lambda x: lemmatizacion(x))

df_es=df_es.drop(["words", "Feedback"], axis=1)
df_es.to_csv("../Raw Datas/Feedback_Lematized_es.csv")

In [23]:
###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_es["frecwords"]=df_es["lemma"].apply(lambda x: common_words(x))
df_es=df_es.drop(["lemma"], axis = 1)
df_es.to_csv("../Raw Datas/freqwords_es.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_es.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_spanish.csv")

In [24]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Promoter,Perú - Lima,nan,facil,2
2022,December,Promoter,Perú - Lima,nan,esperando,1
2022,December,Promoter,Perú - Lima,nan,propuesta,1
2022,December,Promoter,Perú - Lima,nan,hector,1
2022,December,Promoter,Perú - Lima,nan,renovar,1


In [25]:
########################### PROCESAMIENTO EN Arabic ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_ar=df[df["Language"]=="ar"]
df_ar["ShopperRegionI"]=df_ar["ShopperRegionI"].map(str)
df_ar["CaseProductGroup"]=df_ar["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_ar['index_2'] = df_ar[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_ar = df_ar.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_ar = df_ar.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_ar = df_ar.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_ar=df_ar.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_ar.head()

Feedback  \
Year Month    index_2                                                                             
2022 December Detractor/Egipt - Cairo/nan     المستثمر معكم ممكن ان يكون في اي  بلد فهل يوجد...   
              Promoter/Egipt - Cairo/domains  جهد مشكور وتفاني في خدمة العملاء الخدمة ممتازة...   
     November Promoter/Egipt - Cairo/email    متعاون ومفيدة جدا خدمة عملاء ممتازة ... سهلت ك...   
              Promoter/Egipt - Cairo/nan          دعم مميز من الاستاذ علي, شخص محترم و فاهم جدا   
              Promoter/Egipt - Cairo/domains  كل شيئ كان جيدا فقط الإتصال كان متقطعا أنا أتص...   

                                                 len  
Year Month    index_2                                 
2022 December Detractor/Egipt - Cairo/nan         85  
              Promoter/Egipt - Cairo/domains  32 125  
     November Promoter/Egipt - Cairo/email     17 83  
              Promoter/Egipt - Cairo/nan          46  
              Promoter/Egipt - Cairo/domains  100 24

In [26]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_ar["Feedback"] = df_ar["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
#def remove_accents(a):
#        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
#df_ar['Feedback'] = df_ar['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_ar["words"] = df_ar["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_ar = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_ar(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_ar["words"] = df_ar["words"].apply(lambda x: remove_stopwords_ar(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_ar["lemma"]=df_ar["words"].apply(lambda x: lemmatizacion(x))

df_ar=df_ar.drop(["words", "Feedback"], axis=1)
df_ar.to_csv("../Raw Datas/Feedback_Lematized_ar.csv",encoding="utf-8-sig")


In [27]:
####### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_ar["frecwords"]=df_ar["lemma"].apply(lambda x: common_words(x))
df_ar=df_ar.drop(["lemma"], axis = 1)
df_ar.to_csv("../Raw Datas/freqwords_ar.csv",encoding="utf-8-sig")


##### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_ar.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_arabic.csv",encoding="utf-8-sig")

In [28]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Detractor,Egipt - Cairo,nan,ممكن,2
2022,December,Detractor,Egipt - Cairo,nan,المستثمر,1
2022,December,Detractor,Egipt - Cairo,nan,معكم,1
2022,December,Detractor,Egipt - Cairo,nan,يكون,1
2022,December,Detractor,Egipt - Cairo,nan,بلد,1


In [29]:
########################## PROCESAMIENTO EN French ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_fr=df[df["Language"]=="fr"]
df_fr["ShopperRegionI"]=df_fr["ShopperRegionI"].map(str)
df_fr["CaseProductGroup"]=df_fr["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_fr['index_2'] = df_fr[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_fr = df_fr.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_fr = df_fr.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_fr = df_fr.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_fr=df_fr.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_fr.head()

Feedback  \
Year Month    index_2                                                                                 
2022 December Promoter/Turkey - Istanbul/domains                                   comment archived   
              Detractor/Perú - Lima/sitebuilder   jai appel pour avoir de laide pour lier mon ag...   
              Detractor/Perú - Lima/hosting       mon appel a t coup aprs 30 lbs sans rponse - l...   
              Promoter/Perú - Lima/sitebuilder    trs bonne qualit du conseil, la personne  bien...   
              Promoter/Perú - Lima/hosting                                 bon service  la clientle   

                                                      len  
Year Month    index_2                                      
2022 December Promoter/Turkey - Istanbul/domains       16  
              Detractor/Perú - Lima/sitebuilder   316 156  
              Detractor/Perú - Lima/hosting       188 257  
              Promoter/Perú - Lima/sitebuilder     154 32  
              Promoter/Perú - Lima/hosting             26

In [30]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_fr["Feedback"] = df_fr["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_fr['Feedback'] = df_fr['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_fr["words"] = df_fr["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_fr = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_fr(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_fr["words"] = df_fr["words"].apply(lambda x: remove_stopwords_fr(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_fr["lemma"]=df_fr["words"].apply(lambda x: lemmatizacion(x))

df_fr=df_fr.drop(["words", "Feedback"], axis=1)
df_fr.to_csv("../Raw Datas/Feedback_Lematized_fr.csv")


In [31]:
###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_fr["frecwords"]=df_fr["lemma"].apply(lambda x: common_words(x))
df_fr=df_fr.drop(["lemma"], axis = 1)
df_fr.to_csv("../Raw Datas/freqwords_fr.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_fr.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_french.csv")

In [32]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Promoter,Turkey - Istanbul,domains,comment,1
2022,December,Promoter,Turkey - Istanbul,domains,archive,1
2022,December,Detractor,Perú - Lima,sitebuilder,avoir,2
2022,December,Detractor,Perú - Lima,sitebuilder,employ,2
2022,December,Detractor,Perú - Lima,sitebuilder,lien,2


In [33]:
########################## PROCESAMIENTO EN German ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_de=df[df["Language"]=="de"]
df_de["ShopperRegionI"]=df_de["ShopperRegionI"].map(str)
df_de["CaseProductGroup"]=df_de["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_de['index_2'] = df_de[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_de = df_de.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_de = df_de.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_de = df_de.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_de=df_de.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_de.head()


Feedback  \
Year Month    index_2                                                                                    
2022 December Promoter/Turkey - Istanbul/domains     die mitarbeiter sind nett. sehr hilfreich und ...   
              Promoter/Perú - Lima/account           perfekt service: lsungsorientiert, professione...   
              Promoter/Turkey - Istanbul/email       mir wurde ausgeholfen. danke ich bin insgesamt...   
              Promoter/Turkey - Istanbul/nan         sehr nett und super freundlich und sind auch s...   
              Passive/Turkey - Istanbul/sitebuilder  sehr geduldig bei einem nicht internetafinen m...   

                                                                              len  
Year Month    index_2                                                              
2022 December Promoter/Turkey - Istanbul/domains     26 39 138 46 111 15 48 51 39  
              Promoter/Perú - Lima/account                                    116  
              Promoter/Turkey - Istanbul/email               28 47 40 32 53 98 29  
              Promoter/Turkey - Istanbul/nan                         59 47 101 58  
              Passive/Turkey - Istanbul/sitebuilder                            61

In [34]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_de["Feedback"] = df_de["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_de['Feedback'] = df_de['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_de["words"] = df_de["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_de = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_de(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_de["words"] = df_de["words"].apply(lambda x: remove_stopwords_de(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_de["lemma"]=df_de["words"].apply(lambda x: lemmatizacion(x))

df_de=df_de.drop(["words", "Feedback"], axis=1)
df_de.to_csv("../Raw Datas/Feedback_Lematized_de.csv")


In [35]:
###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_de["frecwords"]=df_de["lemma"].apply(lambda x: common_words(x))
df_de=df_de.drop(["lemma"], axis = 1)
df_de.to_csv("../Raw Datas/freqwords_de.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_de.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_german.csv")


In [36]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Promoter,Turkey - Istanbul,domains,nett,2
2022,December,Promoter,Turkey - Istanbul,domains,danke,2
2022,December,Promoter,Turkey - Istanbul,domains,account,2
2022,December,Promoter,Turkey - Istanbul,domains,freundlich,2
2022,December,Promoter,Turkey - Istanbul,domains,mitarbeiter,1


In [37]:
########################## PROCESAMIENTO EN Italian ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_it=df[df["Language"]=="it"]
df_it["ShopperRegionI"]=df_it["ShopperRegionI"].map(str)
df_it["CaseProductGroup"]=df_it["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_it['index_2'] = df_it[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_it = df_it.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_it = df_it.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_it = df_it.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_it=df_it.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_it.head()

Feedback  \
Year Month    index_2                                                                           
2022 December Promoter/Perú - Lima/domains  personale molto gentile,  davvero il pi grande...   
              Promoter/Perú - Lima/account  amazing upport personale competente e preparat...   
              Promoter/Perú - Lima/email    professionale, gentile, preparata l'assistente...   
              Passive/Perú - Lima/domains   12 minuti di attesa recuperati dalla disponibi...   
              Promoter/Perú - Lima/nan      il signore che mi ha dato assistenza  stato co...   

                                                            len  
Year Month    index_2                                            
2022 December Promoter/Perú - Lima/domains  57 13 32 36 111 102  
              Promoter/Perú - Lima/account             14 32 31  
              Promoter/Perú - Lima/email          33 104 138 21  
              Passive/Perú - Lima/domains                    96  
              Promoter/Perú - Lima/nan             100 17 16 18

In [38]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_it["Feedback"] = df_it["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_it['Feedback'] = df_it['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_it["words"] = df_it["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_it = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_it(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_it["words"] = df_it["words"].apply(lambda x: remove_stopwords_it(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_it["lemma"]=df_it["words"].apply(lambda x: lemmatizacion(x))

df_it=df_it.drop(["words", "Feedback"], axis=1)
df_it.to_csv("../Raw Datas/Feedback_Lematized_it.csv")


In [39]:
###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_it["frecwords"]=df_it["lemma"].apply(lambda x: common_words(x))
df_it=df_it.drop(["lemma"], axis = 1)
df_it.to_csv("../Raw Datas/freqwords_it.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_it.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_italian.csv")


In [40]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Promoter,Perú - Lima,domains,gentile,3
2022,December,Promoter,Perú - Lima,domains,molto,2
2022,December,Promoter,Perú - Lima,domains,great,2
2022,December,Promoter,Perú - Lima,domains,support,2
2022,December,Promoter,Perú - Lima,domains,stata,2


In [41]:
########################## PROCESAMIENTO EN Turkish ##################################################################

df= pd.read_csv("../Raw Datas/Coment_con_idioma.csv")
df_tr=df[df["Language"]=="tr"]
df_tr["ShopperRegionI"]=df_tr["ShopperRegionI"].map(str)
df_tr["CaseProductGroup"]=df_tr["CaseProductGroup"].map(str)

#Crea columna con el CSAT/lob y borra las otras columnas:
df_tr['index_2'] = df_tr[["NPS","SiteCode","CaseProductGroup"]].apply(lambda x:'/'.join(x), axis = 1)
df_tr = df_tr.drop(["NPS", "Language","SiteCode","CaseProductGroup"], axis = 1)
df
#Crea un index doble, por semana y CSAT/Lob. 
df_tr = df_tr.set_index(["Year","Month", "index_2"]).sort_index(ascending = [True, True]).astype(str)

#Agrupa para que solo se tengan por cada semana, 1 fila de CSAT/lob y aplica transform para unir todo el texto que aplica en una celda. 
df_tr = df_tr.groupby(["Year","Month", "index_2"]).transform(lambda x : ' '.join(x)).drop_duplicates(keep = 'first')
df_tr=df_tr.drop(["Unnamed: 0", "Time","ShopperId","ShopperRegionI","CustomerType","Guide","Supervisor","SurveyId","Orecxid","CssScore","NpsScore"], axis = 1)

df_tr.head()


Feedback  \
Year Month    index_2                                                                                   
2022 December Promoter/Turkey - Istanbul/hosting    benim iin kompleks ve korkutucu olan bir sreci...   
              Detractor/Turkey - Istanbul/hosting   teknik bilgisi ok iyiydi. bir nceki grtm kii m...   
              Detractor/Turkey - Istanbul/security            teknik bilgi yetersiz ya da batan savd.   
              Promoter/Turkey - Istanbul/domains    ilgili alakal bir temsilci teekkrler saolsun a...   
              Promoter/Turkey - Istanbul/email      arkadalar ok yardmc oluyor saolsunlar mteri hi...   

                                                                                                  len  
Year Month    index_2                                                                                  
2022 December Promoter/Turkey - Istanbul/hosting    601 109 117 50 548 59 39 32 237 60 66 190 56 4...  
              Detractor/Turkey - Istanbul/hosting                                              187 87  
              Detractor/Turkey - Istanbul/security                                                 41  
              Promoter/Turkey - Istanbul/domains    39 61 25 48 90 60 17 52 118 77 39 32 32 44 167...  
              Promoter/Turkey - Istanbul/email              42 122 62 102 64 33 36 64 191 34 39 17 74

In [42]:
################## LIMPIEZA DE TEXTO ###########################################################################################33
#Lowercase todo el texto
df_tr["Feedback"] = df_tr["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df_tr['Feedback'] = df_tr['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


# LISTA Numero de palabras:
df_tr["words"] = df_tr["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#common_words_tr = ["the", "a", "to", "if", "is", "it", "of", "and", "or", "an", "as", "i", "me", "my", "não", "sei","we", "our", "ours", "you", "your", "yours", "he", "she", "him", "his", "her", "hers", "its", "they", "them", "’", "de", "o", "their", "what", "which", "who", "whom", "this", "that", "am", "are", "was", "were", "be", "been", "being", "Ã©", "fue", "que",  "have", "has", "had", "do", "does", "did", "but", "at", "by", "with", "from", "here", "when", "where", "how", "don", "all", "any", "both", "each", "few", "more", "some", "such", "no", "nor", "too", "very", "can", "will", "just", "ve",   "on", "in", "for", " ", "i’m", "i'm", "it’s","things",": ","so","not","would", "can", "cannot","�😡","there", "about",   "don't", "now", "also", "make", "take", "these", "didn't", "doing", "cant't","!",".",",", "...", "n't","(",")","wasn't","", "nan"]

#Remover stopwords y puntuacion
def remove_stopwords_tr(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish")))))))       #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words and len(w) > 2]  # code to remove stop words
    return removedstopwords

df_tr["words"] = df_tr["words"].apply(lambda x: remove_stopwords_tr(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df_tr["lemma"]=df_tr["words"].apply(lambda x: lemmatizacion(x))

df_tr=df_tr.drop(["words", "Feedback"], axis=1)
df_tr.to_csv("../Raw Datas/Feedback_Lematized_tr.csv")


In [43]:
###### FREQ WORDS ##############
def common_words(listica2):
    fdist = FreqDist(listica2)
    common_words = fdist.most_common(3000)
    return common_words

df_tr["frecwords"]=df_tr["lemma"].apply(lambda x: common_words(x))
df_tr=df_tr.drop(["lemma"], axis = 1)
df_tr.to_csv("../Raw Datas/freqwords_tr.csv")


#### REPORTE FREQ WORDS ##################

df= pd.read_csv("../Raw Datas/freqwords_tr.csv")
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('),','_'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace(',','-'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('_','),'))
df['frecwords'] = df['frecwords'].apply(lambda x: x.replace('[','').replace(']',''))
df.frecwords = df.frecwords.str.split(',')
df2 = df.explode('frecwords')
df2[['Word', 'Frecuency']] = df2['frecwords'].str.split('-', 1, expand=True)
df2[["NPS","SiteCode","CaseProductGroup"]] = df2['index_2'].str.split('/', 2, expand=True)
del df2['index_2']
del df2['frecwords']
df2['Word'] = df2['Word'].apply(lambda x: x.replace('(','').replace("'",''))
df2['Frecuency'] = df2['Frecuency'].map(str)
df2['Frecuency'] = df2['Frecuency'].apply(lambda x: x.replace(')',''))
df2 = df2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word','Frecuency']]
df2["Word"]=df2["Word"].apply(lambda x: x.strip())
df3=df2.set_index("Year")
df3.to_csv("../Raw Datas/words_turkish.csv")


In [44]:
df3.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency
Year,,,,,,
2022,December,Promoter,Turkey - Istanbul,hosting,iin,8
2022,December,Promoter,Turkey - Istanbul,hosting,teekkr,8
2022,December,Promoter,Turkey - Istanbul,hosting,bir,6
2022,December,Promoter,Turkey - Istanbul,hosting,godaddy,6
2022,December,Promoter,Turkey - Istanbul,hosting,teekkrler,4


In [45]:
#### CONCATENAR DE LOS DOS IDIOMAS: 
df1= pd.read_csv("../Raw Datas/words_english.csv")
df2= pd.read_csv("../Raw Datas/words_spanish.csv")
df3= pd.read_csv("../Raw Datas/words_french.csv")
df4= pd.read_csv("../Raw Datas/words_german.csv")
df5= pd.read_csv("../Raw Datas/words_italian.csv")
df6= pd.read_csv("../Raw Datas/words_turkish.csv")
df7= pd.read_csv("../Raw Datas/words_arabic.csv") #Unmark until data
df1["Language"] = "English"
df2["Language"] = "Spanish"
df3["Language"] = "French"
df4["Language"] = "German"
df5["Language"] = "Italian"
df6["Language"] = "Turkish"
df7["Language"] = "Arabic" #Unmark until data
concat = [df1, df2, df3, df4, df5, df6,df7]
words_final_es_en_pt=pd.concat(concat)
words_final_es_en_pt=words_final_es_en_pt.set_index("Year")
words_final_es_en_pt.to_csv("../Raw Datas/words_en_es_pt.csv",encoding="utf-8-sig")

In [46]:
words_final_es_en_pt.head()

,Month,NPS,SiteCode,CaseProductGroup,Word,Frecuency,Language
Year,,,,,,,
2022,December,Detractor,Perú - Lima,NaN,website,4,English
2022,December,Detractor,Perú - Lima,NaN,call,4,English
2022,December,Detractor,Perú - Lima,NaN,support,4,English
2022,December,Detractor,Perú - Lima,NaN,godaddy,4,English
2022,December,Detractor,Perú - Lima,NaN,speak,3,English


In [47]:
################################ BIGRAMAS INGLES  #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_en.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_en.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_en.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 2, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_en.csv")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Detractor,Perú - Lima,nan,call,back,2
2022,December,Detractor,Perú - Lima,nan,comment,archive,2
2022,December,Detractor,Perú - Lima,nan,true,shit,2
2022,December,Detractor,Perú - Lima,nan,spend,nearly,1
2022,December,Detractor,Perú - Lima,nan,nearly,day,1


In [48]:
################################ BIGRAMAS ESPAÑOL #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_es.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_es.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_es.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 2, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_es.csv")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Promoter,Perú - Lima,nan,esperando,propuesta,1
2022,December,Promoter,Perú - Lima,nan,propuesta,hector,1
2022,December,Promoter,Perú - Lima,nan,hector,renovar,1
2022,December,Promoter,Perú - Lima,nan,renovar,ssl,1
2022,December,Promoter,Perú - Lima,nan,ssl,efeicaz,1


In [49]:
################################ BIGRAMAS Arabic #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_ar.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_ar.csv",encoding="utf-8-sig")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_ar.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 3, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_ar.csv",encoding="utf-8-sig")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Detractor,Egipt - Cairo,nan,المستثمر,معكم,1
2022,December,Detractor,Egipt - Cairo,nan,معكم,ممكن,1
2022,December,Detractor,Egipt - Cairo,nan,ممكن,يكون,1
2022,December,Detractor,Egipt - Cairo,nan,يكون,بلد,1
2022,December,Detractor,Egipt - Cairo,nan,بلد,فهل,1


In [50]:
################################ BIGRAMAS French #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_fr.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_fr.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_fr.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 3, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_fr.csv")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Promoter,Turkey - Istanbul,domains,comment,archive,1
2022,December,Detractor,Perú - Lima,sitebuilder,jai,appel,1
2022,December,Detractor,Perú - Lima,sitebuilder,appel,avoir,1
2022,December,Detractor,Perú - Lima,sitebuilder,avoir,laide,1
2022,December,Detractor,Perú - Lima,sitebuilder,laide,lier,1


In [51]:
################################ BIGRAMAS German #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_de.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_de.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_de.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 3, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_de.csv")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Promoter,Turkey - Istanbul,domains,mitarbeiter,nett,1
2022,December,Promoter,Turkey - Istanbul,domains,nett,hilfreich,1
2022,December,Promoter,Turkey - Istanbul,domains,hilfreich,kompetente,1
2022,December,Promoter,Turkey - Istanbul,domains,kompetente,antworten,1
2022,December,Promoter,Turkey - Istanbul,domains,antworten,anfang,1


In [52]:
################################ BIGRAMAS Italian #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_it.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_it.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_it.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 3, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_it.csv")

df.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Promoter,Perú - Lima,domains,molto,gentile,2
2022,December,Promoter,Perú - Lima,domains,great,support,2
2022,December,Promoter,Perú - Lima,domains,personale,molto,1
2022,December,Promoter,Perú - Lima,domains,gentile,davvero,1
2022,December,Promoter,Perú - Lima,domains,davvero,grande,1


In [53]:
################################ BIGRAMAS Turkish #################################
df=pd.read_csv("../Raw Datas/Feedback_Lematized_tr.csv")
df['lemma'] = df['lemma'].apply(lambda x: x.replace('[','').replace(']',''))
def common_bigramss(listica3):
    lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(lista_nueva))
    threshold = 2
    filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    filtered_bigram_dist = FreqDist(filtered_bigrams)
    return filtered_bigram_dist.most_common(300)

df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df=df.set_index("Year")
df.to_csv("../Raw Datas/FreqBigrams_tr.csv")

####################### REPORTE BIGRAMAS ###################################

fre = pd.read_csv("../Raw Datas/FreqBigrams_tr.csv")
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"),','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('[(',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('"',' '))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('),','_'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace(',','-'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('_','),'))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace('((','('))
fre['bigrams'] = fre['bigrams'].apply(lambda x: x.replace("'-","' -"))
fre.bigrams = fre.bigrams.str.split(',')
fre2 = fre.explode('bigrams')
fre2[['Word1','Word2', 'Frecuency']] = fre2['bigrams'].str.split('-', 2, expand=True)
fre2[["NPS","SiteCode","CaseProductGroup"]] = fre2['index_2'].str.split('/', 3, expand=True)
del fre2['index_2']
del fre2['bigrams']
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace('(','').replace("'",''))
fre2['Word1'] = fre2['Word1'].apply(lambda x: str(x).replace("'",'').replace("'",''))
fre2['Word2'] = fre2['Word2'].apply(lambda x: str(x).replace("'",''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(')',''))
fre2['Frecuency'] = fre2['Frecuency'].apply(lambda x: str(x).replace(']',''))
fre2 = fre2[['Year',"Month","NPS","SiteCode","CaseProductGroup",'Word1','Word2','Frecuency']]
df=fre2.set_index("Year")
df['Word1'] = df['Word1'].apply(lambda x: x.strip())
df['Word2'] = df['Word2'].apply(lambda x: x.strip())
df.to_csv("../Raw Datas/Bigramas_tr.csv")

df.head()


,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency
Year,,,,,,,
2022,December,Promoter,Turkey - Istanbul,hosting,yardmc,oldu,3
2022,December,Promoter,Turkey - Istanbul,hosting,teekkr,ederim,3
2022,December,Promoter,Turkey - Istanbul,hosting,teekkr,ederiz,3
2022,December,Promoter,Turkey - Istanbul,hosting,elinden,gelen,2
2022,December,Promoter,Turkey - Istanbul,hosting,godaddy,nin,2


In [54]:
#### CONCATENAR DE LOS DOS IDIOMAS: 
df1= pd.read_csv("../Raw Datas/Bigramas_en.csv")
df2= pd.read_csv("../Raw Datas/Bigramas_es.csv")
df3= pd.read_csv("../Raw Datas/Bigramas_ar.csv") 
df4= pd.read_csv("../Raw Datas/Bigramas_de.csv")
df5= pd.read_csv("../Raw Datas/Bigramas_it.csv")
df6= pd.read_csv("../Raw Datas/Bigramas_tr.csv")
df7= pd.read_csv("../Raw Datas/Bigramas_fr.csv")
df1["Language"] = "English"
df2["Language"] = "Spanish"
df3["Language"] = "Arabic" 
df4["Language"] = "German"
df5["Language"] = "Italian"
df6["Language"] = "Turkish"
df7["Language"] = "French"
concat = [df1, df2, df3, df4, df5, df6, df7]
words_final_es_en_pt=pd.concat(concat)
words_final_es_en_pt=words_final_es_en_pt.set_index("Year")
words_final_es_en_pt.to_csv("../Raw Datas/Bigramas_en_es_pt.csv",encoding="utf-8-sig")

words_final_es_en_pt.head()

,Month,NPS,SiteCode,CaseProductGroup,Word1,Word2,Frecuency,Language
Year,,,,,,,,
2022,December,Detractor,Perú - Lima,NaN,call,back,2,English
2022,December,Detractor,Perú - Lima,NaN,comment,archive,2,English
2022,December,Detractor,Perú - Lima,NaN,true,shit,2,English
2022,December,Detractor,Perú - Lima,NaN,spend,nearly,1,English
2022,December,Detractor,Perú - Lima,NaN,nearly,day,1,English


In [55]:
###### Lista de Palabras Unicas ################################################

df=pd.read_csv("../Raw Datas/words_en_es_pt.csv")
words=df["Word"]
unique_words=words.unique()
key_unique_words = pd.DataFrame(unique_words)
key_unique_words .rename(columns = {0:'Word'}, inplace = True)
key_unique_words["Word"]= key_unique_words["Word"].map(str)
key_unique_words=key_unique_words["Word"].apply(lambda x: x.strip())
key_unique_words.to_csv("../Raw Datas/KeyUniqueWords_en_es_pt.csv",encoding="utf-8-sig")

key_unique_words.head()

0    website
1       call
2    support
3    godaddy
4      speak
Name: Word, dtype: object

In [56]:
###### Lista de Agentes Unicos ################################################

df=pd.read_csv("../Raw Datas/RAWDATASQL.csv")
words=df["Guide"]
unique_words=words.unique()
key_unique_words = pd.DataFrame(unique_words)
key_unique_words .rename(columns = {0:'Guide'}, inplace = True)
key_unique_words["Guide"]= key_unique_words["Guide"].map(str)
key_unique_words=key_unique_words["Guide"].apply(lambda x: x.strip())
key_unique_words.to_csv("../Raw Datas/KeyUniqueAgents.csv")
key_unique_words.head()

0     Roberto Mechano
1     Estefani Huaman
2    Bryan Malpartida
3       Karim Mohamed
4        Ahmed Masoud
Name: Guide, dtype: object

In [57]:
###### Lista de Unico SubType ################################################

df=pd.read_csv("../Raw Datas/RAWDATASQL.csv")
words=df["ShopperRegionI"]
unique_words=words.unique()
key_unique_words = pd.DataFrame(unique_words)
key_unique_words .rename(columns = {0:'ShopperRegionI'}, inplace = True)
key_unique_words["ShopperRegionI"]= key_unique_words["ShopperRegionI"].map(str)
key_unique_words=key_unique_words["ShopperRegionI"].apply(lambda x: x.strip())
key_unique_words.to_csv("../Raw Datas/KeyUniqueShopperRegionI.csv")
key_unique_words.head()

0    International - Top Markets
1            International - RoW
2                  United States
3                            nan
4          International - China
Name: ShopperRegionI, dtype: object

In [58]:
#Reading Data
import nltk
from nltk.probability import FreqDist
# nltk.download('punkt')
# nltk.download('wordnet')
import pandas as pd
import datetime as dt
from datetime import datetime as dt
import numpy as np
from collections import OrderedDict
pd.options.mode.chained_assignment = None
import unicodedata
import regex as re
from nltk.stem import WordNetLemmatizer
from langdetect import detect
from textblob import TextBlob

In [59]:
#LEER DATA
df=pd.read_csv("../Raw Datas/RAWDATASQL.csv")
#df=df.dropna()

In [60]:
#############################PREPARACION DE DF ###############################################################
#df.rename(columns={'CaseSubType':'SubType','CaseType':'Type'}, inplace=True)
#df.head()

In [61]:
df['Feedback'] = df['Feedback'].map(str)
df=df.drop("Unnamed: 0", axis = 1)
df.columns

Index(['Date', 'Time', 'ShopperId', 'CustomerType', 'Guide', 'Supervisor',
       'ShopperRegionI', 'SurveyId', 'Orecxid', 'CaseProductGroup', 'CssScore',
       'NpsScore', 'NPS', 'Feedback', 'SiteCode', 'Jomax', 'len'],
      dtype='object')

In [62]:
#OBTENER EL MES
df["Date"]=pd.to_datetime(df['Date'])
df["Month"]=df.Date.dt.month_name()
df["Year"]=df.Date.dt.year
#df=df.drop("Date", axis=1)

In [63]:
############### LIMPIEZA DE TEXTO EN FEEDBACK ##########################
#Lowercase: 
df["Feedback"] = df["Feedback"].apply(lambda x: str(x).lower())

#FUNCION PARA REMOVER CARACTERES RAROS
def remove_accents(a):
        return unidecode.unidecode(a.decode('utf-8'))

#Se aplica la funcion a la columna
df['Feedback'] = df['Feedback'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))


#Remover puntuacion
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace(".", ""))
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace(",", ""))
df['Feedback'] = df['Feedback'].apply(lambda x: x.replace("!", ""))
df=df.set_index("Year")

df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax,len,Month
Year,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,roberto3014,78,June
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-09-19,04:15:29,143122891,International Independent,Gonzalo JimenezCaceres,"Quispe Llontop, Alisson",International - RoW,201130921,96935148.0,account,10,10,Promoter,estremamente professionale e gentile,Perú - Lima,gonzalo2385,36,September
2022,2022-09-19,00:20:55,387504276,International Independent,Ahmet Kara,"Basturk, Emin",International - RoW,201130380,96927563.0,security,10,10,Promoter,comment archived,Turkey - Istanbul,ahmet7242,16,September
2022,2022-09-18,23:52:42,178033284,Partner,Rogger GutierrezHuertas,"Cotrina Gomez, Aron",International - RoW,201130386,96926566.0,domains,10,10,Promoter,my experience was very professional and effici...,Perú - Lima,rogger1531,68,September


In [64]:
#LEER DATA ARABIC
df2=pd.read_csv("../Raw Datas/RAWDATASQL.csv")
df2=df2[df2.SiteCode == 'Egipt - Cairo']

df2['Feedback'] = df2['Feedback'].map(str)
df2=df2.drop("Unnamed: 0", axis = 1)

In [65]:
#OBTENER EL MES
df2["Date"]=pd.to_datetime(df2['Date'])
df2["Month"]=df2.Date.dt.month_name()
df2["Year"]=df2.Date.dt.year
############### LIMPIEZA DE TEXTO EN FEEDBACK ##########################
#Lowercase: 
df2["Feedback"] = df2["Feedback"].apply(lambda x: str(x).lower())
#Remover puntuacion
df2['Feedback'] = df2['Feedback'].apply(lambda x: x.replace(".", ""))
df2['Feedback'] = df2['Feedback'].apply(lambda x: x.replace(",", ""))
df2['Feedback'] = df2['Feedback'].apply(lambda x: x.replace("!", ""))
df2=df2.set_index("Year")

In [66]:
df=pd.concat([df, df2])
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax,len,Month
Year,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,roberto3014,78,June
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-09-27,03:13:50,313819021,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201144649,97282115.0,hosting,10,10,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September
2022,2022-09-25,02:26:26,235418468,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141422,97204539.0,email,10,9,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September
2022,2022-09-25,08:05:32,529768147,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141634,97206659.0,NaN,2,1,Detractor,"""no support but you say """" its normal"""" that s...",Egipt - Cairo,abdelrahman0673,88,September


In [67]:
#def get_language(text):
#    try:
#        language = detect(text)
#    except:
#        language = 'no'
#    return language

#def get_language(text):
#    if (text == 'TP Bogota - NA'):
#        return "en"
#    elif (text == 'TP Lima'):
#        return "en"
#    elif (text == 'TP Bogota - ES'):
#        return "es"
#    elif (text == 'TP Bogota - BR'):
#        return "pt"
#    return None

def get_language(text):
    try:
        language = detect(text)
    except:
        language = 'no'
    return language

df["Language"] = df["Feedback"].apply(lambda x: get_language(x))
df['Language'] = df['Language'].replace('en', 'English')
df['Language'] = df['Language'].replace('es', 'Spanish')
df['Language'] = df['Language'].replace('ar', 'Arabic')
df['Language'] = df['Language'].replace('de', 'German')
df['Language'] = df['Language'].replace('it', 'Italian')
df['Language'] = df['Language'].replace('tr', 'Turkish')
df['Language'] = df['Language'].replace('fr', 'French')

#df_idx=df[(df["Language"]!='English')&(df["Language"]!='Spanish')&(df["Language"]!='Portuguese')].index
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax,len,Month,Language
Year,,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,roberto3014,78,June,English
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June,English
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June,English
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-09-27,03:13:50,313819021,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201144649,97282115.0,hosting,10,10,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September,English
2022,2022-09-25,02:26:26,235418468,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141422,97204539.0,email,10,9,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September,English
2022,2022-09-25,08:05:32,529768147,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141634,97206659.0,NaN,2,1,Detractor,"""no support but you say """" its normal"""" that s...",Egipt - Cairo,abdelrahman0673,88,September,English


In [68]:
#df.drop(df_idx, inplace=True)
#df = df.drop(df[df['Language']=='None'].index, inplace=True)
df= df.dropna(subset=["Language"])
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,CssScore,NpsScore,NPS,Feedback,SiteCode,Jomax,len,Month,Language
Year,,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:18:12,282348646,NaN,Roberto Mechano,"Saenz, Mayra",International - Top Markets,201592493,107518616.0,email,0,2,Detractor,happy with product - been trying to pay online...,Perú - Lima,roberto3014,78,June,English
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,10,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June,English
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,10,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,10,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June,English
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,10,10,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-09-27,03:13:50,313819021,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201144649,97282115.0,hosting,10,10,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September,English
2022,2022-09-25,02:26:26,235418468,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141422,97204539.0,email,10,9,Promoter,comment archived,Egipt - Cairo,abdelrahman0673,16,September,English
2022,2022-09-25,08:05:32,529768147,International Independent,Abdelrahman Hashim,"Ashraf, Youstina",International - RoW,201141634,97206659.0,NaN,2,1,Detractor,"""no support but you say """" its normal"""" that s...",Egipt - Cairo,abdelrahman0673,88,September,English


In [69]:
df.Language.value_counts()

English    6857
Turkish     878
Spanish     656
French      353
German      288
Italian     158
no           94
af           91
ca           65
ro           60
da           53
pt           52
nl           48
Arabic       32
id           30
hr           29
cy           23
et           18
sv           18
so           17
sw           14
tl           11
sl            9
cs            8
fi            7
pl            6
lv            6
hu            5
sk            4
sq            2
lt            1
Name: Language, dtype: int64

In [70]:
# LISTA Numero de palabras:
df["words"] = df["Feedback"].apply(lambda x: list(nltk.word_tokenize(x)))

#Remover stopwords y puntuacion
def remove_stopwords(listica):
    from nltk.corpus import stopwords                   
    stop_words = set(list(stopwords.words("english")) + list(stopwords.words("spanish"))
                     + list(stopwords.words('arabic')+ list(stopwords.words("french")
                        + list(stopwords.words("german")+ list(stopwords.words("italian")
                            + list(stopwords.words("turkish"))))))) #Getting a list of stopwords
    removedstopwords = [w for w in listica if not w in stop_words]  # code to remove stop words
    return removedstopwords

df["words"] = df["words"].apply(lambda x: remove_stopwords(x))

#Lemmatizacion: 
def lemmatizacion(listica2):
    WNlemma = nltk.WordNetLemmatizer()       #Getting a list of stopwords
    lemmatized_output = [WNlemma.lemmatize(w, pos = "v") for w in listica2]  # code to remove stop words
    return lemmatized_output

df["lemma"]=df["words"].apply(lambda x: lemmatizacion(x))


df=df.drop(["words"], axis=1)
def common_bigramss(listica3):
    #lista_nueva = listica3.split(',') # Volvemos el string una lista cuyos valores estan separados por ','
    md_bigrams = list(nltk.bigrams(listica3))
    #threshold = 2
    #filtered_bigrams = [bigram for bigram in md_bigrams if len(bigram[0])>threshold and len(bigram[1])>threshold]
    #filtered_bigram_dist = FreqDist(filtered_bigrams)
    return md_bigrams
df["bigrams"]=df["lemma"].apply(lambda x: common_bigramss(x))
df=df.drop("lemma", axis=1)
df["bigrams"]=df["bigrams"].map(str)
df["bigrams"]=df["bigrams"].apply(lambda x: x.replace("'",""))

################################## CATEGORIAS  ##################################################

Categories_list = {"Billing": ["(wait, refund)","(money, back)","(get, money)","(credit, card)","(pay, full)",
                               "(receive, refund)","(get, refund)","(godaddy, refund)","(refund, money)","(pay, godaddy)",
                               "(charge, credit)","(full, refund)","(fee, refund)","(refund, cancel)","(card, charge)",
                               "(money, godaddy)","(refund, godaddy)","(fee, charge)","(refund, service)","(charge, godaddy)",
                               "(get, full)","(card, company)","(confirmation, email)","(rental, property)",
                               "(full, amount)","(email, say)","(give, refund)","(payment, request)",
                               "(security, deposit)","(100, refund)","(get, pay)","(bank, account)",
                               "(insurance, company)","(final, payment)","(damage, deposit)","(refund, issue)",
                               "(deposit, refund)","(issue, refund)","(fee, godaddy)","(rent, house)","(refund, process)",
                               "(refund, property)","(owner, refund)","(esperar, reembolsar)","(dinero, de vuelta)","(obtener, dinero)","(crédito, tarjeta)","(pagar, completo)",
                               "(recibir, reembolsar)","(obtener, reembolsar)","(godaddy, reembolso)","(reembolso, dinero)","(pagar, godaddy)",
                               "(cargo, crédito)","(total, reembolso)","(tarifa, reembolso)","(reembolso, cancelación)","(tarjeta, cargo)",
                               "(dinero, padrino)","(reembolso, padrino)","(tarifa, cargo)","(reembolso, servicio)","(cargo, padrino)",
                               "(obtener, lleno)","(tarjeta, empresa)","(confirmación, email)","(alquiler, propiedad)",
                               "(cantidad total)","(correo electrónico, por ejemplo)","(dar, reembolsar)","(pago, solicitud)",
                               "(seguridad, depósito)","(100, reembolso)","(obtener, pagar)","(banco, cuenta)",
                               "(seguro, compañía)","(final, pago)","(daños, depósito)","(reembolso, emisión)",
                               "(depósito, reembolso)","(emisión, reembolso)","(tarifa, padrino)","(alquiler, casa)","(reembolso, proceso)",
                               "(reembolso, propiedad)","(propietario, reembolso)","(attendre, rembourser)","(argent, retour)","(obtenir, argent)","(crédit, carte)","(payer, plein)",
                               "(recevoir, rembourser)","(obtenir, rembourser)","(parrain, rembourser)","(rembourser, argent)","(payer, parrain)",
                               "(débit, crédit)","(complet, remboursement)","(frais, remboursement)","(remboursement, annulation)","(carte, débit)",
                               "(argent, godaddy)","(remboursement, godaddy)","(frais, charge)","(remboursement, service)","(charge, godaddy)",
                               "(obtenir, complet)","(carte, entreprise)","(confirmation, email)","(location, propriété)",
                               "(total, montant)","(email, say)","(donner, rembourser)","(paiement, demander)",
                               "(sécurité, dépôt)","(100, remboursement)","(obtenir, payer)","(banque, compte)",
                               "(assurance, compagnie)","(final, paiement)","(avarie, caution)","(remboursement, émission)",
                               "(dépôt, remboursement)","(émission, remboursement)","(frais, godaddy)","(location, maison)","(remboursement, processus)",
                               "(remboursement, propriété)","(propriétaire, remboursement)","(warten, erstatten)","(geld, zurück)","(geld bekommen)","(kredit, karte)","(bezahlen, voll)",
                               "(erhalten, erstatten)","(erhalten, erstatten)","(Papa, erstatten)","(Erstattung, Geld)","(bezahlen, Pate)",
                               "(Belastung, Gutschrift)","(Voll, Erstattung)","(Gebühr, Erstattung)","(Erstattung, Stornierung)","(Karte, Belastung)",
                               "(Geld, Pate)","(Rückerstattung, Pate)","(Gebühr, Gebühr)","(Rückerstattung, Service)","(Aufladung, Pate)",
                               "(erhalten, voll)","(Karte, Unternehmen)","(Bestätigung, E-Mail)","(Miete, Eigentum)",
                               "(voller Betrag)","(E-Mail, sagen wir)","(geben, erstatten)","(Zahlung, Anfrage)",
                               "(Sicherheit, Anzahlung)","(100, Rückerstattung)","(Erhalten, Bezahlen)","(Bank, Konto)",
                               "(Versicherung, Firma)","(Abschluss, Zahlung)","(Schaden, Kaution)","(Rückerstattung, Ausgabe)",
                               "(Kaution, Erstattung)","(Ausstellung, Erstattung)","(Gebühr, Pate)","(Miete, Haus)","(Erstattung, Bearbeitung)",
                               "(Erstattung, Eigentum)","(Eigentümer, Erstattung)","(aspetta, rimborso)","(soldi, indietro)","(ricevi, denaro)","(credito, carta)","(paga, intero)",
                               "(ricevere, rimborso)","(ottenere, rimborso)","(godaddy, rimborso)","(rimborso, denaro)","(pagare, godaddy)",
                               "(addebito, credito)","(completo, rimborso)","(commissione, rimborso)","(rimborso, annullamento)","(carta, addebito)",
                               "(soldi, godaddy)","(rimborso, godaddy)","(commissione, addebito)","(rimborso, servizio)","(addebito, godaddy)",
                               "(get, full)","(card, company)","(confirmation, email)","(affitto, proprietà)",
                               "(intero, importo)","(e-mail, dire)","(give, rimborso)","(pagamento, richiesta)",
                               "(sicurezza, deposito)","(100, rimborso)","(ottenere, pagare)","(banca, conto)",
                               "(assicurazione, compagnia)","(finale, pagamento)","(danno, deposito)","(rimborso, emissione)",
                               "(deposito, rimborso)","(emissione, rimborso)","(commissione, godaddy)","(affitto, casa)","(rimborso, processo)",
                               "(rimborso, proprietà)","(proprietario, rimborso)","(bekle, para iadesi)","(para, geri)","(al, para)","(kredi, kart)","(öde, tam)",
                               "(alma, para iadesi)","(al, para iadesi)","(godaddy, para iadesi)","(para iadesi, para)","(ödeme, godaddy)",
                               "(ücret, kredi)","(tam, para iadesi)","(ücret, para iadesi)","(para iadesi, iptal)","(kart, ücret)",
                               "(para, godaddy)","(para iadesi, godaddy)","(ücret, ücret)","(para iadesi, hizmet)","(ücret, godaddy)",
                               "(al, dolu)","(kart, şirket)","(onay, e-posta)","(kiralama, mülk)",
                               "(dolu, miktar)","(e-posta, söyle)","(ver, geri ödeme)","(ödeme, talep)",
                               "(güvenlik, depozito)","(100, geri ödeme)","(al, öde)","(banka, hesap)",
                               "(sigorta, şirket)","(nihai, ödeme)","(hasar, depozito)","(para iadesi, ihraç)",
                               "(depozito, para iadesi)","(sorun, para iadesi)","(ücret, godaddy)","(kira, ev)","(para iadesi, işlem)",
                               "(para iadesi, mülk)","(sahip, para iadesi)"],

                   "Call Issues": ["(person, talk)","(try, contact)","(godaddy, contact)","(live, person)","(long, time)",
                                   "(several, time)","(put, hold)","(time, long)","(call, customer)","(three, time)",
                                   "(back, call)","(call, back)","(phone, call)","(someone, else)","(call, time)",
                                   "(call, drop)","(minutes, call)","(wait, hold)","(wait, time)","(contact, godaddy)",
                                   "(time, phone)","(get, phone)","(speak, supervisor)","(take, call)","(number, call)",
                                   "(need, call)","(hold, minutes)","(someone, answer)","(send, email)","(get, call)",
                                   "(call, godaddy)","(long, wait)","(hold, hour)","(minute, wait)","(hang, call)","(tell, contact)",
                                   "(get, back)","(call, disconnect)","(shorter, wait)","(come, back)","(wait, hear)",
                                   "(speak, someone)","(time, wait)","(hear, back)","(never, hear)","(need, contact)",
                                   "(disconnect, call)","(never, call)","(time, call)","(get, disconnect)","(email, godaddy)",
                                   "(wait, phone)","(phone, number)","(take, long)","(find, phone)","(someone, call)",
                                   "(call, get)","(receive, email)","(back, get)","(wait, long)","(tell, call)","(answer, phone)",
                                   "(receive, call)","(hold, time)","(end, call)","(live, agent)","(give, phone)","(hold, long)",
                                   "(call, wait)","(minutes, wait)","(answer, call)","(phone, line)","(talk, someone)",
                                   "(minutes, hold)","(call, answer)","(contact, customer)","(call, twice)","(second, call)",
                                   "(try, chat)","(agent, disconnect)","(need, speak)","(get, touch)","(minutes, get)",
                                   "(online, chat)","(disconnect, chat)","(wait, speak)","(get, email)","(chat, function)",
                                   "(speak, representative)","(time, hold)","(call, issue)","(time, chat)","(long, hold)",
                                   "(try, call)","(live, chat)","(godaddy, call)","(less, wait)","(phone, wait)",
                                   "(get, representative)","(time, speak)","(call, hold)","(take, minutes)","(use, chat)",
                                   "(chat, service)","(chat, feature)","(godaddy, chat)","(minutes, chat)","(persona, hablar)","(intentar, contactar)","(godaddy, contactar)","(vivir, persona)","(largo, tiempo)",
                                   "(varios, tiempo)","(poner, mantener)","(tiempo, largo)","(llamar, cliente)","(tres, tiempo)",
                                   "(volver, llamar)","(llamar, volver)","(teléfono, llamar)","(alguien, más)","(llamar, hora)",
                                   "(llamar, dejar)","(minutos, llamar)","(esperar, esperar)","(esperar, tiempo)","(contactar, padrino)",
                                   "(hora, teléfono)","(obtener, teléfono)","(hablar, supervisor)","(tomar, llamar)","(número, llamar)",
                                   "(necesito, llamar)","(esperar, minutos)","(alguien, responder)","(enviar, correo electrónico)","(obtener, llamar)",
                                   "(llamar, padrino)","(largo, esperar)","(esperar, hora)","(minuto, esperar)","(colgar, llamar)","(decir, contactar)",
                                   "(volver, volver)","(llamar, desconectar)","(más corto, esperar)","(volver, volver)","(esperar, oír)",
                                   "(hablar, alguien)","(tiempo, esperar)","(escuchar, volver)","(nunca, escuchar)","(necesitar, contactar)",
                                   "(desconectar, llamar)","(nunca, llamar)","(hora, llamar)","(obtener, desconectar)","(email, padrino)",
                                   "(esperar, teléfono)","(teléfono, número)","(tomar, largo)","(buscar, teléfono)","(alguien, llamar)",
                                   "(llamar, obtener)","(recibir, correo electrónico)","(volver, obtener)","(esperar, largo)","(decir, llamar)","(responder, teléfono)",
                                   "(recibir, llamar)","(esperar, tiempo)","(terminar, llamar)","(en vivo, agente)","(dar, teléfono)","(esperar, largo)",
                                   "(llamar, esperar)","(minutos, esperar)","(responder, llamar)","(teléfono, línea)","(hablar, alguien)",
                                   "(minutos, espera)","(llamada, respuesta)","(contacto, cliente)","(llamada, dos veces)","(segundo, llamada)",
                                   "(probar, chatear)","(agente, desconectar)","(necesitar, hablar)","(obtener, tocar)","(minutos, obtener)",
                                   "(en línea, chatear)","(desconectar, chatear)","(esperar, hablar)","(obtener, correo electrónico)","(chatear, función)",
                                   "(hablar, representante)","(tiempo, espera)","(llamada, problema)","(hora, chat)","(largo, espera)",
                                   "(intenta, llama)","(vive, chatea)","(godaddy, llama)","(menos, espera)","(teléfono, espera)",
                                   "(obtener, representante)","(hora, hablar)","(llamar, esperar)","(tomar, minutos)","(usar, chatear)",
                                   "(chat, servicio)","(chat, función)","(godaddy, chat)","(minutos, chat)","(kişi, konuşma)","(dene, iletişim)","(godaddy, iletişim)","(canlı, kişi)","(uzun, zaman)",
                                   "(birkaç, zaman)","(koy, beklet)","(süre, uzun)","(çağrı, müşteri)","(üç, zaman)",
                                   "(geri arama, arama)","(arama, geri arama)","(telefon, arama)","(birisi, başkası)","(arama, zaman)",
                                   "(ara, bırak)","(dakika, ara)","(bekle, bekle)","(bekle, zaman)","(kontak, godaddy)",
                                   "(zaman, telefon)","(al, telefon)","(konuş, süpervizör)","(al, ara)","(numara, ara)",
                                   "(gerekli, ara)","(beklet, dakika)","(birisi, cevapla)","(gönder, e-posta)","(al, ara)",
                                   "(ara, godaddy)","(uzun, bekle)","(beklet, saat)","(dakika, bekle)","(bekle, ara)","(anlat, temas kur)",
                                   "(al, geri dön)","(ara, bağlantıyı kes)","(daha kısa, bekle)","(gel, geri dön)","(bekle, duy)",
                                   "(konuş, biri)","(zaman, bekle)","(duy, geri dön)","(asla, duy)","(ihtiyaç, temas)",
                                   "(bağlantıyı kes, ara)","(asla, ara)","(zaman, ara)","(al, bağlantıyı kes)","(e-posta, godaddy)",
                                   "(bekle, telefon)","(telefon, numara)","(al, uzun)","(bul, telefon)","(birisi, ara)",
                                   "(ara, al)","(al, e-posta)","(geri al)","(bekle, uzun süre)","(söyle, ara)","(cevap, telefon)",
                                   "(al, ara)","(beklet, zaman)","(son, ara)","(canlı, temsilci)","(ver, telefon et)","(beklet, uzun)",
                                   "(ara, bekle)","(dakika, bekle)","(cevap, ara)","(telefon, hat)","(konuş, biri)",
                                   "(dakika, beklet)","(ara, cevapla)","(kontak, müşteri)","(ara, iki kez)","(saniye, ara)",
                                   "(dene, sohbet et)","(aracı, bağlantıyı kes)","(ihtiyaç duy, konuş)","(al, dokun)","(dakika, al)",
                                   "(çevrimiçi, sohbet)","(bağlantıyı kes, sohbet et)","(bekle, konuş)","(al, e-posta)","(sohbet, işlev)",
                                   "(konuş, temsilci)","(zaman, beklet)","(çağrı, sorun)","(zaman, sohbet)","(uzun, beklet)",
                                   "(dene, ara)","(canlı, sohbet)","(godaddy, ara)","(daha az, bekle)","(telefon, bekle)",
                                   "(al, temsilci)","(zaman, konuş)","(ara, bekle)","(al, dakika)","(kullan, sohbet et)",
                                   "(sohbet, servis)","(sohbet, özellik)","(godaddy, sohbet)","(dakika, sohbet)","(persona, conversazione)","(provare, contatto)","(godaddy, contatto)","(vivere, persona)","(lungo, tempo)",
                                   "(più, tempo)","(metti, tieni)","(tempo, lungo)","(chiamata, cliente)","(tre, tempo)",
                                   "(indietro, chiamata)","(chiamata, indietro)","(telefono, chiamata)","(qualcuno, altro)","(chiamata, ora)",
                                   "(call, drop)","(minutes, call)","(wait, hold)","(wait, time)","(contact, godaddy)",
                                   "(ora, telefono)","(ricevi, telefono)","(parla, supervisore)","(rispondi, chiama)","(numero, chiama)",
                                   "(necessità, chiamata)","(attesa, minuti)","(qualcuno, risposta)","(invio, e-mail)","(ricevi, chiamata)",
                                   "(call, Godaddy)","(long, wait)","(hold, hour)","(minute, wait)","(hang, call)","(tell, contact)",
                                   "(ottenere, tornare)","(chiamare, disconnettere)","(più breve, attendere)","(tornare, tornare)","(aspettare, ascoltare)",
                                   "(parla, qualcuno)","(tempo, aspetta)","(ascolta, torna)","(mai, ascolta)","(bisogno, contatto)",
                                   "(disconnetti, chiama)","(mai, chiama)","(tempo, chiama)","(ricevi, disconnetti)","(e-mail, godaddy)",
                                   "(aspetta, telefono)","(telefono, numero)","(prendi, lungo)","(trova, telefono)","(qualcuno, chiama)",
                                   "(call, get)","(receive, email)","(back, get)","(wait, long)","(tell, call)","(answer, phone)",
                                   "(ricezione, chiamata)","(attesa, ora)","(fine, chiamata)","(dal vivo, agente)","(dare, telefono)","(attesa, lunga)",
                                   "(chiama, aspetta)","(minuti, aspetta)","(risposta, chiama)","(telefono, linea)","(parla, qualcuno)",
                                   "(minuti, attesa)","(chiamata, risposta)","(contatto, cliente)","(chiamata, due volte)","(secondo, chiamata)",
                                   "(provare, chattare)","(agente, disconnettersi)","(necessità, parlare)","(ottenere, toccare)","(minuti, ottenere)",
                                   "(online, chat)","(disconnetti, chat)","(wait, speak)","(get, email)","(chat, function)",
                                   "(parla, rappresentante)","(tempo, attesa)","(chiamata, problema)","(tempo, chat)","(tempo, attesa)",
                                   "(try, call)","(live, chat)","(godaddy, call)","(less, wait)","(phone, wait)",
                                   "(ottenere, rappresentante)","(tempo, parlare)","(chiamare, mettere in attesa)","(prendere, minuti)","(usare, chat)",
                                   "(chat, servizio)","(chat, feature)","(godaddy, chat)","(minuti, chat)","(Person, Gespräch)","(Versuch, Kontakt)","(Godaddy, Kontakt)","(Live, Person)","(Lange, Zeit)",
                                   "(mehrere, Zeit)","(setzen, halten)","(Zeit, lang)","(Anruf, Kunde)","(drei, Zeit)",
                                   "(zurück, anruf)","(anruf, zurück)","(telefon, anruf)","(jemand, anderes)","(anruf, uhrzeit)",
                                   "(Anruf, Auflegen)","(Minuten, Anruf)","(Warten, Halten)","(Warten, Zeit)","(Kontakt, Pate)",
                                   "(Zeit, Telefon)","(Holen, Telefon)","(Sprechen, Supervisor)","(Annehmen, Anrufen)","(Nummer, Anrufen)",
                                   "(brauchen, anrufen)","(halten, minuten)","(jemand, antworten)","(senden, email)","(erhalten, anrufen)",
                                   "(Ruf, Pate)","(lang, warten)","(halten, Stunde)","(Minute, warten)","(hängen, anrufen)","(erzählen, kontaktieren)",
                                   "(holen, zurück)","(anrufen, trennen)","(kürzer, warten)","(kommen, zurück)","(warten, hören)",
                                   "(sprechen, jemanden)","(Zeit, warten)","(hören, zurück)","(nie, hören)","(brauchen, kontaktieren)",
                                   "(trennen, anrufen)","(nie, anrufen)","(Zeit, anrufen)","(erhalten, trennen)","(E-Mail, Gott)",
                                   "(warten, telefonieren)","(telefonieren, nummer)","(nehmen, lang)","(finden, telefonieren)","(jemand, anrufen)",
                                   "(anrufen, erhalten)","(empfangen, E-Mail)","(zurück, erhalten)","(warten, lange)","(sagen, anrufen)","(antworten, telefonieren)",
                                   "(empfangen, anruf)","(halten, zeit)","(beenden, anruf)","(live, agent)","(geben, telefonieren)","(halten, lang)",
                                   "(anrufen, warten)","(Minuten, warten)","(Annehmen, Anrufen)","(Telefon, Leitung)","(sprechen, jemand)",
                                   "(Minuten, Halten)","(Anruf, Antwort)","(Kontakt, Kunde)","(Anruf, zweimal)","(Sekunde, Anruf)",
                                   "(versuchen, chatten)","(Agent, trennen)","(brauchen, sprechen)","(erhalten, berühren)","(Minuten, erhalten)",
                                   "(online, chat)","(trennen, chat)","(warten, sprechen)","(abrufen, email)","(chat, funktion)",
                                   "(Sprechen, Vertreter)","(Zeit, Halten)","(Anruf, Ausgabe)","(Zeit, Chat)","(Lang, Halten)",
                                   "(versuchen, anrufen)","(live, chatten)","(Papa, anrufen)","(weniger, warten)","(telefonieren, warten)",
                                   "(erhalten, Vertreter)","(Zeit, sprechen)","(anrufen, halten)","(nehmen, Minuten)","(benutzen, chatten)",
                                   "(chat, service)","(chat, feature)","(godaddy, chat)","(minutes, chat)","(personne, parler)","(essayer, contacter)","(parrain, contacter)","(vivre, personne)","(long, temps)",
                                   "(plusieurs, temps)","(mettre, tenir)","(temps, long)","(appel, client)","(trois, temps)",
                                   "(retour, appel)","(appel, retour)","(téléphone, appel)","(quelqu'un, autre)","(appel, heure)",
                                   "(call, drop)","(minutes, call)","(wait, hold)","(wait, time)","(contact, godaddy)",
                                   "(heure, téléphone)","(obtenir, téléphone)","(parler, superviseur)","(prendre, appeler)","(numéro, appeler)",
                                   "(besoin, appeler)","(attendre, minutes)","(quelqu'un, répondre)","(envoyer, e-mail)","(obtenir, appeler)",
                                   "(call, godaddy)","(long, wait)","(hold, hour)","(minute, wait)","(hang, call)","(tell, contact)",
                                   "(prendre, revenir)","(appeler, déconnecter)","(plus court, attendre)","(revenir, revenir)","(attendre, entendre)",
                                   "(parler, quelqu'un)","(temps, attendre)","(entendre, répondre)","(jamais, entendre)","(besoin, contact)",
                                   "(déconnecter, appeler)","(jamais, appeler)","(heure, appeler)","(obtenir, déconnecter)","(email, godaddy)",
                                   "(attendre, téléphone)","(téléphone, numéro)","(prendre, long)","(trouver, téléphone)","(quelqu'un, appeler)",
                                   "(appeler, obtenir)","(recevoir, envoyer un e-mail)","(retour, obtenir)","(attendre, longtemps)","(dire, appeler)","(répondre, téléphoner)",
                                   "(recevoir, appeler)","(attendre, heure)","(fin, appeler)","(vivre, agent)","(donner, téléphoner)","(attendre, longtemps)",
                                   "(appeler, attendre)","(minutes, attendre)","(répondre, appeler)","(téléphone, ligne)","(parler, quelqu'un)",
                                   "(minutes, attendre)","(appeler, répondre)","(contact, client)","(appeler, deux fois)","(seconde, appeler)",
                                   "(essayer, discuter)","(agent, déconnecter)","(besoin, parler)","(obtenir, toucher)","(minutes, obtenir)",
                                   "(en ligne, chat)","(déconnecter, discuter)","(attendre, parler)","(obtenir, e-mail)","(chat, fonction)",
                                   "(parler, représentant)","(heure, attendre)","(appeler, émettre)","(heure, discuter)","(long, attendre)",
                                   "(essayer, appeler)","(vivre, discuter)","(godaddy, appeler)","(moins, attendre)","(téléphoner, attendre)",
                                   "(obtenir, représentant)","(temps, parler)","(appeler, attendre)","(prendre, minutes)","(utiliser, discuter)",
                                   "(chat, service)","(chat, fonctionnalité)","(godaddy, chat)","(minutes, chat)"], 

                   "Customer Service":["(help, resolve)","(customer, service)","(customer, support)","(service, experience)",
                                       "(kind, helpful)","(quickly, thank)","(service, receive)","(prompt, service)",
                                       "(best, customer)","(great, job)","(helpful, resolve)","(outstanding, customer)",
                                       "(thank, much)","(customer, care)","(service, amaze)","(satisfy, thank)","(thank, help)",
                                       "(experience, thank)","(greatly, appreciate)","(amaze, support)","(good, service)",
                                       "(support, team)","(great, customer)","(thank, take)","(service, eve)","(great, service)",
                                       "(great, thank)","(help, receive)","(nice, helpful)","(support, get)","(good, customer)",
                                       "(service, good)","(efficient, service)","(thank, santiago)","(service, always)",
                                       "(go, beyond)","(awesome, customer)","(much, help)","(helpful, friendly)",
                                       "(support, excellent)","(well, do)","(service, excellent)","(service, great)",
                                       "(excellent, service)","(make, easy)","(service, thank)","(many, thank)",
                                       "(extremely, helpful)","(quickly, efficiently)","(good, thank)","(much, appreciate)",
                                       "(thank, maria)","(thank, assistance)","(excellent, customer)","(great, help)",
                                       "(appreciate, help)","(thank, thank)","(quick, thank)","(really, appreciate)",
                                       "(wonderful, customer)","(excellent, thank)","(helpful, thank)","(helpful, polite)",
                                       "(friendly, helpful)","(helpful, kind)","(quality, support)","(fantastic, service)",
                                       "(thank, juan)","(happy, customer)","(support, thank)","(excellent, response)",
                                       "(thank, good)","(easy, thank)","(quick, efficient)","(support, quick)","(work, thank)",
                                       "(much, great)","(great, support)","(thank, excellent)","(satisfy, customer)",
                                       "(quick, answer)","(helpful, understand)","(thank, great)","(help, thank)","(amaze, customer)",
                                       "(much, thank)","(happy, thank)","(receive, excellent)","(time, thank)","(super, helpful)",
                                       "(super, quick)","(excellent, support)","(exceptional, customer)","(thank, customer)",
                                       "(issue, thank)","(great, work)","(fast, easy)","(service, support)","(great, fast)",
                                       "(fast, friendly)","(extremely, satisfy)","(thank, helpful)","(thank, make)","(efficient, thank)",
                                       "(quick, easy)","(receive, quick)","(exceptional, service)","(pleasantly, surprise)",
                                       "(highly, recommend)","(response, great)","(really, helpful)","(fast, efficient)","(service, happy)",
                                       "(help, quick)","(kind, thank)","(happy, service)","(impress, customer)","(appreciate, thank)",
                                       "(service, helpful)","(help, great)","(satisfy, support)","(service, prompt)","(helpful, excellent)",
                                       "(support, great)","(thank, appreciate)","(thank, happy)","(helpful, fast)","(thank, response)",
                                       "(quick, response)","(amaze, service)","(first, class)","(customer, thank)","(beyond, expectations)",
                                       "(appreciate, quick)","(service, really)","(appreciate, fast)","(really, good)",
                                       "(awesome, thank)","(fantastic, customer)","(service, fast)","(prompt, helpful)",
                                       "(support, receive)","(thank, prompt)","(reply, thank)","(service, super)","(fast, helpful)",
                                       "(good, support)","(quickly, great)","(response, good)","(quick, service)","(quick, respond)",
                                       "(helpful, quick)","(excellent, quick)","(quick, helpful)","(much, quick)","(happy, new)",
                                       "(happy, support)","(really, fast)","(thank, support)","(helpful, prompt)","(process, easy)","(response, helpful)","(great, quick)",
                                       "(service, quick)","(future, thank)","(super, service)","(service, definitely)","(thank, lot)",
                                       "(really, happy)","(helpful, support)","(aide, résolution)","(client, service)","(client, support)","(service, expérience)",
                                       "(gentil, serviable)","(rapidement, merci)","(service, recevoir)","(invite, service)",
                                       "(meilleur, client)","(excellent, travail)","(serviable, résolu)","(exceptionnel, client)",
                                       "(merci, beaucoup)","(client, attention)","(service, étonner)","(satisfaire, remercier)","(remercier, aider)",
                                       "(expérience, remerciement)","(beaucoup, appréciation)","(émerveillement, soutien)","(bon, service)",
                                       "(assistance, équipe)","(super, client)","(merci, prendre)","(service, veille)","(super, service)",
                                       "(super, merci)","(aider, recevoir)","(gentil, serviable)","(soutenir, obtenir)","(bon, client)",
                                       "(service, bon)","(efficace, service)","(merci, santiago)","(service, toujours)",
                                       "(aller, au-delà)","(génial, client)","(beaucoup, aider)","(serviable, amical)",
                                       "(support, excellent)","(eh bien, faites)","(service, excellent)","(service, excellent)",
                                       "(excellent, service)","(faire, facile)","(service, merci)","(beaucoup, merci)",
                                       "(extrêmement, utile)","(rapidement, efficacement)","(bien, merci)","(beaucoup, apprécie)",
                                       "(merci, maria)","(merci, aide)","(excellent, client)","(génial, aide)",
                                       "(apprécier, aider)","(remercier, remercier)","(rapide, remercier)","(vraiment, apprécier)",
                                       "(merveilleux, client)","(excellent, merci)","(serviable, merci)","(serviable, poli)",
                                       "(amical, serviable)","(serviable, gentil)","(qualité, soutien)","(fantastique, service)",
                                       "(merci, juan)","(heureux, client)","(soutien, merci)","(excellent, réponse)",
                                       "(merci, bien)","(facile, merci)","(rapide, efficace)","(soutien, rapide)","(travailler, remercier)",
                                       "(beaucoup, génial)","(génial, soutien)","(merci, excellent)","(satisfait, client)",
                                       "(rapide, réponse)","(utile, comprendre)","(merci, super)","(aide, merci)","(étonner, client)",
                                       "(beaucoup, merci)","(heureux, merci)","(recevoir, excellent)","(temps, remercier)","(super, utile)",
                                       "(super, rapide)","(excellent, support)","(exceptionnel, client)","(merci, client)",
                                       "(problème, merci)","(super, travail)","(rapide, facile)","(service, support)","(super, rapide)",
                                       "(rapide, amical)","(extrêmement, satisfait)","(remercier, utile)","(remercier, faire)","(efficace, remercier)",
                                       "(rapide, facile)","(recevoir, rapide)","(exceptionnel, service)","(agréablement, surprise)",
                                       "(fortement, recommande)","(réponse, super)","(vraiment, utile)","(rapide, efficace)","(service, content)",
                                       "(aide, rapide)","(gentil, merci)","(heureux, service)","(impressionner, client)","(apprécier, remercier)",
                                       "(service, serviable)","(aide, super)","(satisfait, support)","(service, prompt)","(serviable, excellent)",
                                       "(soutenir, super)","(remercier, apprécier)","(remercier, heureux)","(utile, rapide)","(remercier, réponse)",
                                       "(rapide, réponse)","(étonner, service)","(première, classe)","(client, remercier)","(au-delà des attentes)",
                                       "(apprécier, rapide)","(service, vraiment)","(apprécier, rapide)","(vraiment, bien)",
                                       "(génial, merci)","(fantastique, client)","(service, rapide)","(rapide, utile)",
                                       "(soutenir, recevoir)","(remercier, demander)","(répondre, remercier)","(service, super)","(rapide, utile)",
                                       "(bon, support)","(rapidement, super)","(réponse, bien)","(rapide, service)","(rapide, réponse)",
                                       "(utile, rapide)","(excellent, rapide)","(rapide, utile)","(beaucoup, rapide)","(heureux, nouveau)",
                                       "(heureux, soutien)","(vraiment, rapide)","(merci, soutien)","(utile, rapide)","(processus, facile)","(réponse, utile)","( super, rapide)",
                                       "(service, rapide)","(futur, merci)","(super, service)","(service, certainement)","(merci, beaucoup)",
                                       "(vraiment, heureux)","(serviable, soutien)","(Hilfe, Lösung)","(Kunde, Service)","(Kunde, Support)","(Service, Erfahrung)",
                                       "(freundlich, hilfreich)","(schnell, danke)","(dienst, erhalten)","(aufforderung, dienst)",
                                       "(bester, Kunde)","(großartig, Arbeit)","(hilfreich, entschlossen)","(herausragend, Kunde)",
                                       "(vielen Dank)","(Kunde, Fürsorge)","(Service, Erstaunen)","(zufrieden, danke)","(Dank, Hilfe)",
                                       "(Erfahrung, danke)","(großartig, zu schätzen)","(erstaunlich, Unterstützung)","(gut, Service)",
                                       "(support, team)","(great, customer)","(thank, take)","(service, eve)","(great, service)",
                                       "(großartig, danke)","(helfen, erhalten)","(nett, hilfreich)","(unterstützen, erhalten)","(gut, Kunde)",
                                       "(Service, gut)","(effizient, Service)","(Danke, Santiago)","(Service, immer)",
                                       "(gehen, darüber hinaus)","(super, Kunde)","(viel, Hilfe)","(hilfreich, freundlich)",
                                       "(Unterstützung, ausgezeichnet)","(na gut)","(Service, ausgezeichnet)","(Service, großartig)",
                                       "(ausgezeichnet, Service)","(machen, einfach)","(Service, danke)","(viele, danke)",
                                       "(extrem, hilfreich)","(schnell, effizient)","(gut, danke)","(viel, danke)",
                                       "(Danke, Maria)","(Danke, Hilfe)","(Ausgezeichnet, Kunde)","(Großartig, Hilfe)",
                                       "(schätzen, helfen)","(danke, danke)","(schnell, danke)","(wirklich, danke)",
                                       "(wundervoll, Kunde)","(ausgezeichnet, danke)","(hilfreich, danke)","(hilfreich, höflich)",
                                       "(freundlich, hilfsbereit)","(hilfreich, freundlich)","(Qualität, Support)","(fantastisch, Service)",
                                       "(danke, juan)","(zufrieden, kunde)","(unterstützung, danke)","(ausgezeichnet, antwort)",
                                       "(danke, gut)","(einfach, danke)","(schnell, effizient)","(support, schnell)","(arbeit, danke)",
                                       "(viel, großartig)","(großartig, Unterstützung)","(danke, ausgezeichnet)","(zufrieden, Kunde)",
                                       "(schnell, antwort)","(hilfreich, verstehe)","(danke, super)","(hilfe, danke)","(erstaunlich, kunde)",
                                       "(viel, danke)","(glücklich, danke)","(erhalten, ausgezeichnet)","(Zeit, danke)","(super, hilfreich)",
                                       "(super, schnell)","(ausgezeichnet, Support)","(außergewöhnlich, Kunde)","(danke, Kunde)",
                                       "(Problem, danke)","(großartig, funktioniert)","(schnell, einfach)","(Service, Support)","(großartig, schnell)",
                                       "(schnell, freundlich)","(sehr zufrieden)","(danke, hilfreich)","(danke, mache)","(effizient, danke)",
                                       "(schnell, einfach)","(erhalten, schnell)","(außergewöhnlich, Service)","(angenehm, Überraschung)",
                                       "(sehr empfehlenswert)","(Antwort, großartig)","(wirklich, hilfreich)","(schnell, effizient)","(Service, zufrieden)",
                                       "(Hilfe, schnell)","(freundlich, danke)","(glücklich, Service)","(beeindrucken, Kunde)","(schätzen, danke)",
                                       "(Service, hilfreich)","(Hilfe, großartig)","(zufrieden, Unterstützung)","(Service, prompt)","(hilfreich, ausgezeichnet)",
                                       "(support, großartig)","(danke, danke)","(danke, glücklich)","(hilfreich, schnell)","(danke, antwort)",
                                       "(schnell, antwort)","(erstaunlich, service)","(erstklassig)","(kunde, danke)","(übertreffen, erwartungen)",
                                       "(schätzen, schnell)","(Service, wirklich)","(schätzen, schnell)","(wirklich, gut)",
                                       "(super, danke)","(fantastisch, Kunde)","(Service, schnell)","(schnell, hilfreich)",
                                       "(support, erhalten)","(danke, prompt)","(antwort, danke)","(service, super)","(schnell, hilfreich)",
                                       "(gut, Unterstützung)","(schnell, großartig)","(Antwort, gut)","(schnell, Service)","(schnell, Antwort)",
                                       "(hilfreich, schnell)","(ausgezeichnet, schnell)","(schnell, hilfreich)","(viel, schnell)","(glücklich, neu)",
                                       "(glücklich, Unterstützung)","(wirklich, schnell)","(danke, Unterstützung)","(hilfreich, prompt)","(Prozess, einfach)","(Antwort, hilfreich)","( toll, schnell)",
                                       "(Service, schnell)","(Zukunft, danke)","(super, Service)","(Service, auf jeden Fall)","(danke, viel)",
                                       "(wirklich, glücklich)","(hilfreich, unterstützend)","(aiuto, risoluzione)","(cliente, servizio)","(cliente, supporto)","(servizio, esperienza)",
                                       "(gentile, disponibile)","(rapidamente, grazie)","(servizio, ricezione)","(prompt, servizio)",
                                       "(migliore, cliente)","(ottimo, lavoro)","(utile, risoluto)","(eccezionale, cliente)",
                                       "(grazie mille)","(cliente, cura)","(servizio, stupisci)","(soddisfatto, grazie)","(grazie, aiuto)",
                                       "(esperienza, grazie)","(molto, apprezzo)","(stupisci, supporto)","(buono, servizio)",
                                       "(supporto, team)","(ottimo, cliente)","(grazie, prendi)","(servizio, vigilia)","(ottimo, servizio)",
                                       "(ottimo, grazie)","(aiuto, ricevere)","(gentile, utile)","(supporto, ottenere)","(bene, cliente)",
                                       "(servizio, buono)","(efficiente, servizio)","(grazie, santiago)","(servizio, sempre)",
                                       "(vai, oltre)","(fantastico, cliente)","(molto, aiuto)","(disponibile, amichevole)",
                                       "(supporto, eccellente)","(bene, fai)","(servizio, eccellente)","(servizio, ottimo)",
                                       "(eccellente, servizio)","(make, easy)","(servizio, grazie)","(molti, grazie)",
                                       "(estremamente utile)","(velocemente, efficientemente)","(bene, grazie)","(molto, apprezzato)",
                                       "(grazie, maria)","(grazie, assistenza)","(eccellente, cliente)","(ottimo, aiuto)",
                                       "(apprezzare, aiutare)","(grazie, grazie)","(veloce, grazie)","(davvero, apprezzo)",
                                       "(meraviglioso, cliente)","(eccellente, grazie)","(utile, grazie)","(utile, gentile)",
                                       "(amichevole, disponibile)","(utile, gentile)","(qualità, supporto)","(fantastico, servizio)",
                                       "(grazie, juan)","(felice, cliente)","(supporto, grazie)","(eccellente, risposta)",
                                       "(grazie, bene)","(facile, grazie)","(veloce, efficiente)","(supporto, veloce)","(lavoro, grazie)",
                                       "(molto, fantastico)","(ottimo, supporto)","(grazie, eccellente)","(soddisfatto, cliente)",
                                       "(veloce, risposta)","(utile, comprensibile)","(grazie, fantastico)","(aiuto, grazie)","(stupisci, cliente)",
                                       "(molto, grazie)","(felice, grazie)","(ricezione, eccellente)","(tempo, grazie)","(super, utile)",
                                       "(super, veloce)","(eccellente, supporto)","(eccezionale, cliente)","(grazie, cliente)",
                                       "(problema, grazie)","(ottimo, lavoro)","(veloce, facile)","(servizio, supporto)","(ottimo, veloce)",
                                       "(veloce, amichevole)","(estremamente soddisfatto)","(grazie, disponibile)","(grazie, fai)","(efficiente, grazie)",
                                       "(veloce, facile)","(ricevere, veloce)","(eccezionale, servizio)","(piacevolmente, sorpresa)",
                                       "(altamente consigliato)","(risposta, ottimo)","(davvero, utile)","(veloce, efficiente)","(servizio, felice)",
                                       "(aiuto, veloce)","(gentile, grazie)","(felice, servizio)","(impressione, cliente)","(apprezzare, ringraziare)",
                                       "(servizio, utile)","(aiuto, ottimo)","(soddisfatto, supporto)","(servizio, tempestivo)","(utile, eccellente)",
                                       "(supporto, fantastico)","(grazie, apprezzo)","(grazie, felice)","(utile, veloce)","(grazie, risposta)",
                                       "(rapido, risposta)","(stupore, servizio)","(prima, classe)","(cliente, grazie)","(oltre, aspettative)",
                                       "(apprezzare, veloce)","(servizio, davvero)","(apprezzare, veloce)","(davvero, bene)",
                                       "(fantastico, grazie)","(fantastico, cliente)","(servizio, veloce)","(pronto, utile)",
                                       "(supporto, ricezione)","(grazie, tempestivo)","(risposta, grazie)","(servizio, super)","(veloce, disponibile)",
                                       "(buono, supporto)","(velocemente, ottimo)","(risposta, buono)","(veloce, servizio)","(veloce, rispondi)",
                                       "(utile, veloce)","(eccellente, veloce)","(veloce, utile)","(molto, veloce)","(felice, nuovo)",
                                       "(felice, supporto)","(davvero, veloce)","(grazie, supporto)","(utile, pronto)","(processo, facile)","(risposta, utile)","( ottimo, veloce)",
                                       "(servizio, veloce)","(futuro, grazie)","(super, servizio)","(servizio, decisamente)","(grazie, lotto)",
                                       "(davvero, felice)","(utile, supporto)","(yardım, çözüm)","(müşteri, hizmet)","(müşteri, destek)","(hizmet, deneyim)",
                                       "(nazik, yardımcı)","(hızlı, teşekkürler)","(hizmet, alma)","(istem, hizmet)",
                                       "(en iyi, müşteri)","(harika, iş)","(yardımcı, çözüm)","(olağanüstü, müşteri)",
                                       "(teşekkür, çok)","(müşteri, ilgi)","(hizmet, şaşırtma)","(tatmin et, teşekkür et)","(teşekkür, yardım et)",
                                       "(deneyim, teşekkür ederim)","(çok, minnettarım)","(şaşırtma, destek)","(iyi, hizmet)",
                                       "(destek, ekip)","(harika, müşteri)","(teşekkür, al)","(hizmet, arife)","(harika, hizmet)",
                                       "(harika, teşekkürler)","(yardım, al)","(güzel, yardımcı)","(destek, al)","(iyi, müşteri)",
                                       "(hizmet, iyi)","(verimli, hizmet)","(teşekkürler, santiago)","(hizmet, her zaman)",
                                       "(git, öteye git)","(harika, müşteri)","(çok, yardım et)","(yardımcı, arkadaş canlısı)",
                                       "(destek, mükemmel)","(iyi, yap)","(servis, mükemmel)","(servis, harika)",
                                       "(mükemmel, hizmet)","(yap, kolay)","(hizmet, teşekkürler)","(çok, teşekkürler)",
                                       "(son derece, yardımcı)","(hızlı, verimli)","(iyi, teşekkür ederim)","(çok, minnettarım)",
                                       "(teşekkür, maria)","(teşekkür, yardım)","(mükemmel, müşteri)","(harika, yardım et)",
                                       "(takdir et, yardım et)","(teşekkürler, teşekkürler)","(çabuk, teşekkürler)","(gerçekten, teşekkürler)",
                                       "(harika, müşteri)","(mükemmel, teşekkür ederim)","(yardımcı, teşekkür ederim)","(yardımcı, kibar)",
                                       "(arkadaş canlısı, yardımsever)","(yardımcı, nazik)","(kalite, destek)","(harika, hizmet)",
                                       "(teşekkür, juan)","(mutlu, müşteri)","(destek, teşekkür)","(mükemmel, yanıt)",
                                       "(teşekkürler, iyi)","(kolay, teşekkürler)","(hızlı, verimli)","(destek, çabuk)","(çalış, teşekkürler)",
                                       "(çok, harika)","(harika, destek)","(teşekkür, mükemmel)","(memnun, müşteri)",
                                       "(hızlı, cevapla)","(yardımcı, anladım)","(teşekkür, harika)","(yardım, teşekkür et)","(şaşırt, müşteri)",
                                       "(çok, teşekkürler)","(mutlu, teşekkürler)","(al, mükemmel)","(zaman, teşekkürler)","(süper, yardımcı)",
                                       "(süper, hızlı)","(mükemmel, destek)","(olağanüstü, müşteri)","(teşekkür, müşteri)",
                                       "(sorun, teşekkür)","(harika, iş)","(hızlı, kolay)","(servis, destek)","(harika, hızlı)",
                                       "(hızlı, arkadaş canlısı)","(son derece, tatmin edici)","(teşekkür, yardımcı)","(teşekkür, yap)","(etkili, teşekkür et)",
                                       "(hızlı, kolay)","(alma, hızlı)","(olağanüstü, hizmet)","(hoş, sürpriz)",
                                       "(kesinlikle tavsiye ederim)","(yanıt, harika)","(gerçekten, yararlı)","(hızlı, verimli)","(hizmet, mutlu)",
                                       "(yardım, çabuk)","(nazik, teşekkür)","(mutlu, hizmet)","(etkilemek, müşteri)","(takdir, teşekkür et)",
                                       "(servis, yardımcı)","(yardım, harika)","(tatmin et, destek)","(servis, istemi)","(yardımcı, mükemmel)",
                                       "(destek, harika)","(teşekkürler, teşekkürler)","(teşekkürler, mutlu)","(yardımcı, hızlı)","(teşekkürler, yanıtlar)",
                                       "(hızlı, yanıt)","(şaşırtma, hizmet)","(birinci sınıf)","(müşteri, teşekkür)","(beklentilerin ötesinde)",
                                       "(takdir, hızlı)","(hizmet, gerçekten)","(takdir, hızlı)","(gerçekten, iyi)",
                                       "(harika, teşekkürler)","(harika, müşteri)","(hizmet, hızlı)","(hızlı, yardımcı)",
                                       "(destek, al)","(teşekkür, istemi)","(yanıt ver, teşekkür et)","(hizmet, süper)","(hızlı, yardımcı)",
                                       "(iyi, destek)","(hızlı, harika)","(yanıt, iyi)","(hızlı, servis)","(hızlı, yanıt ver)",
                                       "(yardımcı, hızlı)","(mükemmel, hızlı)","(hızlı, yardımcı)","(çok, hızlı)","(mutlu, yeni)",
                                       "(mutlu, destek)","(gerçekten, hızlı)","(teşekkür, destek)","(yardımcı, hızlı)","(süreç, kolay)","(yanıt, yardımcı)","( harika, hızlı)",
                                       "(hizmet, hızlı)","(gelecek, teşekkür)","(süper, hizmet)","(hizmet, kesinlikle)","(teşekkür, çok)",
                                       "(gerçekten, mutlu)","(yardımcı, destek)","(ayuda, resolver)","(cliente, servicio)","(cliente, soporte)","(servicio, experiencia)",
                                       "(amable, útil)","(rápidamente, gracias)","(servicio, recibir)","(prompt, servicio)",
                                       "(mejor, cliente)","(excelente, trabajo)","(útil, resuelto)","(excelente, cliente)",
                                       "(muchas gracias)","(cliente, atención)","(servicio, sorprender)","(satisfacer, gracias)","(gracias, ayuda)",
                                       "(experiencia, gracias)","(muy apreciado)","(asombrado, apoyo)","(bien, servicio)",
                                       "(soporte, equipo)","(genial, cliente)","(gracias, toma)","(servicio, eve)","(genial, servicio)",
                                       "(excelente, gracias)","(ayuda, recibir)","(agradable, útil)","(apoyo, obtener)","(bien, cliente)",
                                       "(servicio, bueno)","(eficiente, servicio)","(gracias, santiago)","(servicio, siempre)",
                                       "(ir, más allá)","(increíble, cliente)","(mucha, ayuda)","(útil, amable)",
                                       "(soporte, excelente)","(bueno, hazlo)","(servicio, excelente)","(servicio, excelente)",
                                       "(excelente, servicio)","(hacer, fácil)","(servicio, gracias)","(muchas, gracias)",
                                       "(extremadamente útil)","(rápidamente, eficientemente)","(bien, gracias)","(mucho, aprecio)",
                                       "(gracias, maria)","(gracias, ayuda)","(excelente, cliente)","(genial, ayuda)",
                                       "(aprecio, ayuda)","(gracias, gracias)","(rápido, gracias)","(realmente, aprecio)",
                                       "(maravilloso, cliente)","(excelente, gracias)","(útil, gracias)","(útil, educado)",
                                       "(amable, útil)","(útil, amable)","(calidad, apoyo)","(fantástico, servicio)",
                                       "(gracias, juan)","(feliz, cliente)","(apoyo, gracias)","(excelente, respuesta)",
                                       "(gracias, bien)","(fácil, gracias)","(rápido, eficiente)","(apoyo, rápido)","(trabajo, gracias)",
                                       "(mucho, genial)","(genial, apoyo)","(gracias, excelente)","(satisfacer, cliente)",
                                       "(respuesta rápida)","(útil, entender)","(gracias, genial)","(ayuda, gracias)","(sorprende, cliente)",
                                       "(mucho, gracias)","(feliz, gracias)","(recibir, excelente)","(tiempo, gracias)","(muy útil)",
                                       "(súper, rápido)","(excelente, apoyo)","(excepcional, cliente)","(gracias, cliente)",
                                       "(problema, gracias)","(excelente, trabajo)","(rápido, fácil)","(servicio, soporte)","(excelente, rápido)",
                                       "(rápido, amigable)","(extremadamente, satisfecho)","(gracias, útil)","(gracias, hacer)","(eficiente, gracias)",
                                       "(rápido, fácil)","(recibir, rápido)","(excepcional, servicio)","(gratamente, sorpresa)",
                                       "(muy recomendable)","(respuesta, genial)","(muy útil)","(rápido, eficiente)","(servicio, contento)",
                                       "(ayuda, rápido)","(amable, gracias)","(feliz, servicio)","(impresionar, cliente)","(aprecio, gracias)",
                                       "(servicio, útil)","(ayuda, genial)","(satisfacer, apoyo)","(servicio, aviso)","(útil, excelente)",
                                       "(apoyo, genial)","(gracias, aprecio)","(gracias, feliz)","(útil, rápido)","(gracias, respuesta)",
                                       "(respuesta rápida)","(asombrar, servicio)","(primera clase)","(cliente, gracias)","(más allá de las expectativas)",
                                       "(aprecio, rápido)","(servicio, realmente)","(aprecio, rápido)","(realmente, bueno)",
                                       "(increíble, gracias)","(fantástico, cliente)","(servicio, rápido)","(rápido, útil)",
                                       "(apoyo, recibir)","(gracias, pronto)","(responder, gracias)","(servicio, super)","(rápido, útil)",
                                       "(bueno, apoyo)","(rápido, genial)","(respuesta, bueno)","(rápido, servicio)","(rápido, responder)",
                                       "(útil, rápido)","(excelente, rápido)","(rápido, útil)","(mucho, rápido)","(feliz, nuevo)",
                                       "(feliz, apoyo)","(realmente, rápido)","(gracias, apoyo)","(útil, rápido)","(proceso, fácil)","(respuesta, útil)","( genial, rápido)",
                                       "(servicio, rápido)","(futuro, gracias)","(super, servicio)","(servicio, definitivamente)","(muchas gracias)",
                                       "(realmente, feliz)","(útil, apoyo)"],

                   "Issue Resolution": ["(fix, problem)","(solve, problem)","(answer, question)","(nothing, could)",
                                        "(could, answer)","(get, answer)","(never, get)","(godaddy, still)","(issue, resolve)",
                                        "(wait, see)","(help, find)","(find, another)","(resolve, issue)","(unable, resolve)",
                                        "(rep, could)","(wait, response)","(helpful, issue)","(try, help)","(get, response)",
                                        "(help, resolve)","(issue, godaddy)","(tell, wait)","(get, resolve)","(able, answer)",
                                        "(waste, time)","(try, get)","(take, care)","(could, find)","(get, help)","(could, do)",
                                        "(everything, could)","(get, issue)","(able, help)","(problem, resolve)",
                                        "(problem, still)","(fully, resolve)","(resolve, matter)","(resolve, yet)",
                                        "(could, resolve)","(nothing, resolve)","(issue, never)","(still, wait)",
                                        "(question, answer)","(resolve, question)","(could, get)","(never, resolve)",
                                        "(tell, could)","(days, later)","(ask, question)","(could, help)","(help, issue)",
                                        "(never, receive)","(help, problem)","(many, time)","(able, resolve)","(issue, call)",
                                        "(help, get)","(helpful, resolve)","(still, unresolved)","(completely, resolve)",
                                        "(issue, completely)","(try, resolve)","(resolve, thank)","(problem, godaddy)",
                                        "(case, number)","(still, get)","(business, days)","(every, time)","(solve, problems)",
                                        "(still, resolve)","(receive, response)","(issue, still)","(fix, issue)",
                                        "(get, information)","(godaddy, never)","(explain, issue)","(timely, manner)","(able, get)",
                                        "(address, issue)","(resolve, owner)","(easier, find)","(receive, confirmation)",
                                        "(still, receive)","(time, issue)","(work, issue)","(two, weeks)","(absolutely, nothing)",
                                        "(resolution, issue)","(hard, find)","(find, answer)","(address, problem)","(third, time)",
                                        "(resolve, problem)","(go, beyond)","(care, issue)","(resolve, quickly)","(second, time)",
                                        "(problem, quickly)","(time, get)","(please, help)","(tech, support)","(need, fix)",
                                        "(issue, instead)","(help, need)","(extremely, difficult)","(let, know)","(nothing, wait)",
                                        "(solve, issue)","(representative, answer)","(handle, issue)","(resolve, godaddy)",
                                        "(take, time)","(difficult, find)","(appreciate, help)","(issue, thank)","(issue, need)",
                                        "(answer, issue)","(helpful, answer)","(issue, nothing)","(issue, quickly)",
                                        "(response, time)","(difficult, get)","(technical, issue)","(finally, get)","(problem, solve)",
                                        "(resolve, representative)","(quickly, resolve)","(problem, fix)","(issue, yet)",
                                        "(yet, resolve)","(resolve, customer)","(issue, try)","(issue, representative)","(issue, help)",
                                        "(within, hours)","(wait, hours)","(arreglar, problema)","(resolver, problema)","(respuesta, pregunta)","(nada, podría)",
                                        "(podría, responder)","(obtener, responder)","(nunca, obtener)","(godaddy, todavía)","(problema, resolver)",
                                        "(esperar, ver)","(ayuda, encontrar)","(encontrar, otro)","(resolver, problema)","(no se puede, resolver)",
                                        "(representante, podría)","(esperar, responder)","(útil, problema)","(intentar, ayudar)","(obtener, responder)",
                                        "(ayuda, resolver)","(problema, padrino)","(decir, esperar)","(obtener, resolver)","(capaz, responder)",
                                        "(perder, tiempo)","(intentar, obtener)","(tomar, cuidar)","(podría, encontrar)","(obtener, ayudar)","(podría, hacer)",
                                        "(todo, podría)","(obtener, problema)","(capaz, ayudar)","(problema, resolver)",
                                        "(problema, todavía)","(totalmente, resolver)","(resolver, asunto)","(resolver, todavía)",
                                        "(podría, resolver)","(nada, resolver)","(problema, nunca)","(todavía, esperar)",
                                        "(pregunta, respuesta)","(resolver, pregunta)","(podría, obtener)","(nunca, resolver)",
                                        "(decir, podría)","(días, después)","(preguntar, preguntar)","(podría, ayudar)","(ayuda, problema)",
                                        "(nunca, recibir)","(ayuda, problema)","(muchas veces)","(capaz, resolver)","(problema, llamada)",
                                        "(ayuda, obtener)","(útil, resolver)","(todavía, sin resolver)","(completamente, resolver)",
                                        "(problema, completamente)","(intentar, resolver)","(resolver, agradecer)","(problema, padrino)",
                                        "(caso, número)","(todavía, obtener)","(negocios, días)","(cada, vez)","(resolver, problemas)",
                                        "(aún, resolver)","(recibir, responder)","(emitir, aún)","(arreglar, problema)",
                                        "(obtener, información)","(godaddy, nunca)","(explicar, emitir)","(oportuno, forma)","(poder, obtener)",
                                        "(dirección, problema)","(resolver, dueño)","(más fácil, encontrar)","(recibir, confirmar)",
                                        "(todavía, recibir)","(hora, asunto)","(trabajo, asunto)","(dos, semanas)","(absolutamente, nada)",
                                        "(resolución, problema)","(difícil, encontrar)","(buscar, responder)","(dirección, problema)","(tercero, tiempo)",
                                        "(resolver, problema)","(ir, más allá)","(cuidado, problema)","(resolver, rápidamente)","(segundo, tiempo)",
                                        "(problema, rápido)","(tiempo, obtener)","(por favor, ayuda)","(tecnología, soporte)","(necesita, arreglo)",
                                        "(problema, en cambio)","(ayuda, necesidad)","(extremadamente, difícil)","(informar, saber)","(nada, esperar)",
                                        "(resolver, problema)","(representante, responder)","(manejar, problema)","(resolver, padrino)",
                                        "(tomar, tiempo)","(difícil, encontrar)","(apreciar, ayudar)","(problema, agradecer)","(problema, necesidad)",
                                        "(respuesta, problema)","(útil, respuesta)","(problema, nada)","(problema, rápido)",
                                        "(respuesta, tiempo)","(difícil, obtener)","(técnico, problema)","(finalmente, obtener)","(problema, resolver)",
                                        "(resolver, representante)","(rápidamente, resolver)","(problema, arreglar)","(problema, todavía)",
                                        "(todavía, resolver)","(resolver, cliente)","(problema, probar)","(problema, representante)","(problema, ayuda)",
                                        "(dentro de, horas)","(esperar, horas)","(düzelt, sorun)","(çöz, sorun)","(cevap, soru)","(hiçbir şey, yapamaz)",
                                        "(olabilir, cevaplayabilir)","(al, cevapla)","(asla, al)","(godaddy, hala)","(sorun, çöz)",
                                        "(bekle, gör)","(yardım, bul)","(bul, başka bir tane)","(çöz, sorun)","(yapamıyor, çöz)",
                                        "(tekrar, olabilir)","(bekle, yanıtla)","(yardımcı, sorun)","(dene, yardım et)","(al, yanıt)",
                                        "(yardım, çöz)","(sorun, godaddy)","(söyle, bekle)","(al, çöz)","(mümkün, cevapla)",
                                        "(boşa, zaman)","(dene, al)","(al, ilgilen)","(yapabilir, bulabilir)","(al, yardım et)","(yapabilir, yapabilir)",
                                        "(her şey, olabilir)","(al, sorun)","(mümkün, yardım et)","(problem, çöz)",
                                        "(sorun, hala)","(tamamen, çöz)","(çöz, madde)","(henüz çöz)",
                                        "(olabilir, çözebilir)","(hiçbir şey, çözebilir)","(sorun, asla)","(hala, bekle)",
                                        "(soru, cevap)","(çöz, soru)","(olabilir, alabilir)","(asla, çöz)",
                                        "(söyle, olabilir)","(günler, sonra)","(sor, soru)","(olabilir, yardım edebilir)","(yardım, sorun)",
                                        "(asla, alma)","(yardım, sorun)","(çok, zaman)","(mümkün, çöz)","(sorun, ara)",
                                        "(yardım, al)","(yardımcı, çöz)","(hala, çözülmedi)","(tamamen çöz)",
                                        "(sorun, tamamen)","(dene, çöz)","(çöz, teşekkür et)","(sorun, godaddy)",
                                        "(durum, sayı)","(hala, al)","(iş, günler)","(her zaman, zaman)","(çöz, problemler)",
                                        "(hala, çöz)","(al, yanıtla)","(sorun, dur)","(düzelt, sorun)",
                                        "(al, bilgi)","(godaddy, asla)","(açıkla, sorun)","(zamanında, şekilde)","(mümkün, al)",
                                        "(adres, sorun)","(çöz, sahip)","(daha kolay, bul)","(al, onayla)",
                                        "(hala, alma)","(zaman, sayı)","(iş, konu)","(iki, hafta)","(kesinlikle, hiçbir şey)",
                                        "(çözüm, sorun)","(zor, bul)","(bul, cevapla)","(adres, sorun)","(üçüncü, zaman)",
                                        "(çöz, sorun)","(ileriye git)","(önemseme, sorun)","(çöz, çabuk)","(saniye, zaman)",
                                        "(problem, çabuk)","(zaman, al)","(lütfen, yardım et)","(teknoloji, destek)","(ihtiyaç, düzeltme)",
                                        "(sorun yerine)","(yardım, ihtiyaç)","(son derece, zor)","(bil, bil)","(hiçbir şey, bekle)",
                                        "(çöz, sorun)","(temsilci, cevap)","(işle, sorun)","(çöz, godaddy)",
                                        "(alın, zaman ayırın)","(zor, bul)","(takdir et, yardım et)","(sorun, teşekkür et)","(sorun, ihtiyaç)",
                                        "(cevap, sorun)","(yardımcı, cevap)","(sorun, hiçbir şey yok)","(sorun, hızlı)",
                                        "(yanıt, zaman)","(zor, elde et)","(teknik, sorun)","(sonunda al)","(problem, çöz)",
                                        "(çöz, temsili)","(hızlı, çöz)","(sorun, düzelt)","(sorun, henüz)",
                                        "(henüz, çöz)","(çöz, müşteri)","(sorun, dene)","(sorun, temsilci)","(sorun, yardım)",
                                        "(saat içinde)","(bekle, saat)","(risolvere, problema)","(risolvere, problema)","(risposta, domanda)","(niente, potrebbe)",
                                        "(potrebbe, rispondere)","(ottenere, rispondere)","(mai, ottenere)","(godaddy, ancora)","(problema, risoluzione)",
                                        "(aspetta, vedi)","(aiuto, trova)","(trova, un altro)","(risolvi, problema)","(non in grado di risolvere)",
                                        "(rap, potrebbe)","(aspetta, risposta)","(utile, problema)","(prova, aiuto)","(ricevi, risposta)",
                                        "(aiuto, risoluzione)","(problema, godaddy)","(tell, wait)","(ottenere, risolvere)","(capace, risposta)",
                                        "(perdita, tempo)","(provare, ottenere)","(prendere, curare)","(potrebbe, trovare)","(ottenere, aiutare)","(potrebbe, fare)",
                                        "(tutto, potrebbe)","(ottenere, problema)","(in grado, aiutare)","(problema, risoluzione)",
                                        "(problema, ancora)","(completamente, risolvi)","(risolvere, questione)","(risolvere, ancora)",
                                        "(potrebbe, risolvere)","(niente, risolvere)","(problema, mai)","(ancora, attendere)",
                                        "(domanda, risposta)","(risolvere, domanda)","(potrebbe, ottenere)","(mai, risolvere)",
                                        "(tell, could)","(days, later)","(ask, question)","(potrebbe, help)","(help, issue)",
                                        "(mai, ricevere)","(aiuto, problema)","(molti, tempo)","(capace, risolvere)","(problema, chiamata)",
                                        "(aiuto, ottieni)","(utile, risolvi)","(ancora, irrisolto)","(completamente, risolvi)",
                                        "(problema, completamente)","(prova, risolvi)","(risolvi, grazie)","(problema, papà)",
                                        "(caso, numero)","(ancora, ottenere)","(affari, giorni)","(ogni, volta)","(risolvere, problemi)",
                                        "(ancora, risoluzione)","(ricezione, risposta)","(problema, ancora)","(correzione, problema)",
                                        "(ottenere, informazioni)","(godaddy, mai)","(spiegare, problema)","(tempestivo, modo)","(capace, ottenere)",
                                        "(indirizzo, problema)","(risolvere, proprietario)","(più facile, trovare)","(ricevere, conferma)",
                                        "(ancora, ricevere)","(tempo, problema)","(lavoro, problema)","(due, settimane)","(assolutamente, niente)",
                                        "(risoluzione, problema)","(difficile, trova)","(trova, risposta)","(indirizzo, problema)","(terzo, ora)",
                                        "(risolvere, problema)","(andare, oltre)","(cura, problema)","(risolvere, rapidamente)","(secondo, tempo)",
                                        "(problema, rapidamente)","(tempo, ottenere)","(per favore, aiuto)","(tecnologia, supporto)","(necessità, correzione)",
                                        "(problema, invece)","(aiuto, bisogno)","(estremamente difficile)","(facci sapere)","(niente, aspetta)",
                                        "(risolvere, problema)","(rappresentante, risposta)","(maneggiare, problema)","(risolvere, godaddy)",
                                        "(prendere, tempo)","(difficile, trovare)","(apprezzare, aiutare)","(problema, ringraziare)","(problema, bisogno)",
                                        "(risposta, problema)","(utile, risposta)","(problema, niente)","(problema, rapidamente)",
                                        "(risposta, tempo)","(difficile, ottenere)","(tecnico, problema)","(finalmente, ottenere)","(problema, risolvere)",
                                        "(risolvere, rappresentante)","(velocemente, risolvere)","(problema, risolvere)","(problema, ancora)",
                                        "(ancora, risolvi)","(risolvi, cliente)","(problema, prova)","(problema, rappresentante)","(problema, aiuto)",
                                        "(entro, ore)","(aspetta, ore)","(beheben, problem)","(lösen, problem)","(antwort, frage)","(nichts, könnte)",
                                        "(könnte, antworten)","(bekommen, antworten)","(nie, bekomm)","(godaddy, still)","(problem, lösen)",
                                        "(warten, sehen)","(helfen, finden)","(anderes finden)","(lösen, Problem)","(nicht möglich, lösen)",
                                        "(Vertreter, könnte)","(warten, Antwort)","(hilfreich, Problem)","(versuchen, helfen)","(erhalten, Antwort)",
                                        "(helfen, lösen)","(Problem, Pate)","(sagen, warten)","(holen, lösen)","(fähig, antworten)",
                                        "(Zeitverschwendung)","(versuchen, bekommen)","(nehmen, pflegen)","(könnte, finden)","(bekommen, helfen)","(könnte, tun)",
                                        "(alles, könnte)","(bekommen, ausgeben)","(können, helfen)","(problem, lösen)",
                                        "(Problem, immer noch)","(vollständig, Lösung)","(Lösung, Angelegenheit)","(Lösung, noch)",
                                        "(könnte, lösen)","(nichts, lösen)","(Problem, nie)","(noch, warten)",
                                        "(Frage, Antwort)","(Lösung, Frage)","(könnte, bekommen)","(nie, Lösung)",
                                        "(erzählen, könnten)","(Tage, später)","(fragen, fragen)","(könnte, helfen)","(helfen, ausstellen)",
                                        "(nie, erhalten)","(Hilfe, Problem)","(viele, Zeit)","(fähig, lösen)","(Problem, Anruf)",
                                        "(helfen, bekommen)","(hilfreich, lösen)","(immer noch, ungelöst)","(vollständig, lösen)",
                                        "(Problem, vollständig)","(versuchen, lösen)","(lösen, danke)","(Problem, Gottvater)",
                                        "(Fall, Nummer)","(noch, erhalten)","(Geschäft, Tage)","(jedes Mal)","(Probleme lösen)",
                                        "(immer noch, lösen)","(empfangen, Antwort)","(Problem, immer noch)","(beheben, Problem)",
                                        "(Informationen erhalten)","(Godaddy, nie)","(erklären, herausgeben)","(rechtzeitig, Art und Weise)","(fähig, erhalten)",
                                        "(Adresse, Ausgabe)","(Lösung, Eigentümer)","(einfacher, finden)","(empfangen, Bestätigung)",
                                        "(noch, erhalten)","(Zeit, Ausgabe)","(Arbeit, Ausgabe)","(zwei, Wochen)","(absolut, nichts)",
                                        "(Auflösung, Problem)","(schwer, finden)","(finden, beantworten)","(Adresse, Problem)","(dritte Zeit)",
                                        "(Problem lösen)","(weitergehen)","(Pflege, Problem)","(schnell lösen)","(zweites Mal)",
                                        "(Problem, schnell)", "(Zeit, bekommen)", "(Bitte, Hilfe)", "(Technik, Support)", "(Notwendigkeit, Lösung)",
                                        "(Problem, stattdessen)","(Hilfe, Notwendigkeit)","(extrem, schwierig)","(mitteilen, wissen)","(nichts, warten)",
                                        "(lösen, Problem)","(Vertreter, Antwort)","(handhaben, Problem)","(lösen, Pate)",
                                        "(nehmen, zeit)","(schwierig, finden)","(schätzen, helfen)","(problem, danken)","(problem, brauchen)",
                                        "(Antwort, Problem)","(hilfreich, Antwort)","(Problem, nichts)","(Problem, schnell)",
                                        "(Antwort, Zeit)","(schwierig, bekommen)","(technisch, Problem)","(endlich bekommen)","(Problem, lösen)",
                                        "(Lösung, Vertreter)","(Schnell, Lösung)","(Problem, Lösung)","(Problem noch)",
                                        "(noch, lösen)","(lösen, Kunde)","(Problem, Versuch)","(Problem, Vertreter)","(Problem, Hilfe)",
                                        "(innerhalb, Stunden)","(warten, Stunden)","(réparer, problème)","(résoudre, problème)","(réponse, question)","(rien, pourrait)",
                                        "(pourrait, répondre)","(obtenir, répondre)","(jamais, obtenir)","(mon Dieu, encore)","(problème, résoudre)",
                                        "(attendre, voir)","(aider, trouver)","(trouver, un autre)","(résoudre, problème)","(impossible, résoudre)",
                                        "(rep, could)","(wait, response)","(helpful, issue)","(try, help)","(get, response)",
                                        "(aider, résoudre)","(problème, godaddy)","(dire, attendre)","(obtenir, résoudre)","(capable, répondre)",
                                        "(gaspiller, temps)","(essayer, obtenir)","(prendre soin)","(pourrait, trouver)","(obtenir, aider)","(pourrait, faire)",
                                        "(tout, pourrait)","(obtenir, problème)","(capable, aider)","(problème, résoudre)",
                                        "(problème, encore)","(complètement, résoudre)","(résoudre, importer)","(résoudre, encore)",
                                        "(pourrait, résoudre)","(rien, résoudre)","(problème, jamais)","(toujours, attendre)",
                                        "(question, réponse)","(résoudre, question)","(pourrait, obtenir)","(jamais, résoudre)",
                                        "(dire, pourrait)","(jours, plus tard)","(demander, question)","(pourrait, aider)","(aider, problème)",
                                        "(jamais, recevoir)","(aide, problème)","(beaucoup, fois)","(capable, résoudre)","(problème, appeler)",
                                        "(aider, obtenir)","(utile, résoudre)","(toujours, non résolu)","(complètement, résoudre)",
                                        "(problème, complètement)","(essayer, résoudre)","(résoudre, remercier)","(problème, godaddy)",
                                        "(affaire, nombre)","(toujours, obtenir)","(affaires, jours)","(chaque fois)","(résoudre, problèmes)",
                                        "(toujours, résoudre)","(recevoir, répondre)","(problème, toujours)","(réparer, problème)",
                                        "(obtenir, information)","(pardon, jamais)","(expliquer, publier)","(opportun, manière)","(capable, obtenir)",
                                        "(adresse, problème)","(résoudre, propriétaire)","(plus facile, trouver)","(recevoir, confirmation)",
                                        "(toujours, recevoir)","(heure, problème)","(travailler, problème)","(deux, semaines)","(absolument, rien)",
                                        "(résolution, problème)","(difficile, trouver)","(trouver, répondre)","(adresse, problème)","(troisième, temps)",
                                        "(résoudre, problème)","(aller, au-delà)","(soin, problème)","(résoudre, rapidement)","(seconde, temps)",
                                        "(problème, rapidement)","(temps, obtenir)","(s'il vous plaît, aidez)","(tech, support)","(besoin, réparer)",
                                        "(problème, à la place)","(aide, besoin)","(extrêmement, difficile)","(faire savoir)","(rien, attendre)",
                                        "(résoudre, problème)","(représentant, réponse)","(traiter, problème)","(résoudre, godaddy)",
                                        "(prendre, temps)","(difficile, trouver)","(apprécier, aider)","(problème, remercier)","(problème, besoin)",
                                        "(réponse, problème)","(utile, réponse)","(problème, rien)","(problème, rapidement)",
                                        "(réponse, temps)","(difficile, obtenir)","(technique, problème)","(enfin, obtenir)","(problème, résoudre)",
                                        "(résoudre, représentant)","(rapidement, résoudre)","(problème, résoudre)","(problème, encore)",
                                        "(encore, résoudre)","(résoudre, client)","(problème, essayer)","(problème, représentant)","(problème, aide)",
                                        "(dans, heures)","(attendre, heures)"] ,

                   "Language Skills": [#English
                                       "(representative, speak)","(person, speak)","(hard, understand)",
                                       "(could, understand)","(understand, question)","(hard, time)","(get, someone)",
                                       "(speak, english)","(ask, speak)","(understand, problem)","(heavy, accent)",
                                       "(understand, issue)","(easy, understand)","(difficult, understand)","(understand, accent)",
                                       "(hire, people)","(understand, english)","(rep, speak)","(difficulty, understand)",
                                       "(understand, representative)","(understand, godaddy)","(time, understand)","(understand, say)",
                                       "(gentleman, speak)",
                                       #Spanish
                                       "(representante, hablar)","(persona, hablar)","(difícil, entender)",
                                        "(podría, entender)","(entender, pregunta)","(difícil, tiempo)","(conseguir, alguien)",
                                        "(hablar, inglés)","(preguntar, hablar)","(entender, problema)","(pesado, acento)",
                                        "(entender, tema)","(fácil, entender)","(difícil, entender)","(entender, acento)",
                                        "(contratar, gente)","(entender, inglés)","(rep, hablar)","(dificultad, entender)",
                                        "(entender, representante)","(entender, padrino)","(tiempo, entender)","(entender, decir)",
                                        "(señor, hable)",
                                       #Turkish
                                       "(temsilci, konuş)","(kişi, konuş)","(zor, anla)",
                                        "(anlayabilir, anlayabilir)","(anlayabilir, soru)","(zor, zaman)","(birini alabilir)",
                                        "(konuş, ingilizce)","(sor, konuş)","(anla, sorun)","(ağır, aksan)",
                                        "(anlayın, sorun)","(kolay, anla)","(zor, anla)","(anla, aksan)",
                                        "(kiralama, insanlar)","(anla, ingilizce)","(rep, konuş)","(zorluk, anla)",
                                        "(anla, temsil eden)","(anla, godaddy)","(zaman, anla)","(anla, söyle)",
                                        "(beyefendi, konuş)",
                                       #German
                                       "(Vertreter, sprechen)","(Person, sprechen)","(hart, verstehen)",
                                        "(könnte, verstehen)", "(verstehen, fragen)", "(schwer, Zeit)","(bekommen, jemanden)",
                                        "(sprechen, englisch)","(fragen, sprechen)","(verstehen, problem)","(schwer, akzent)",
                                        "(verstehen, Problem)","(einfach, verstehen)","(schwierig, verstehen)","(verstehen, Akzent)",
                                        "(stellen, leute)","(verstehen, englisch)","(rep, sprechen)","(schwierigkeit, verstehen)",
                                        "(verstehen, repräsentativ)","(verstehen, gott)","(zeit, verstehen)","(verstehen, sagen)",
                                        "(Herr, sprechen)",
                                       #French
                                       "(représentant, parler)","(personne, parler)","(dur, comprendre)",
                                        "(pourrait, comprendre)","(comprendre, questionner)","(dur, temps)","(obtenir, quelqu'un)",
                                        "(parler, anglais)","(demander, parler)","(comprendre, problème)","(lourd, accent)",
                                        "(comprendre, problème)","(facile, comprendre)","(difficile, comprendre)","(comprendre, accent)",
                                        "(embaucher, personnes)","(comprendre, anglais)","(représenter, parler)","(difficulté, comprendre)",
                                        "(comprendre, représentant)","(comprendre, godaddy)","(temps, comprendre)","(comprendre, dire)",
                                        "(monsieur, parlez)",
                                       #Italian
                                       "(rappresentante, parlare)","(persona, parlare)","(difficile, capire)",
                                        "(potrebbe, capire)","(capire, domanda)","(difficile, tempo)","(prendere, qualcuno)",
                                        "(parla, inglese)","(chiedi, parla)","(capire, problema)","(forte, accento)",
                                        "(capire, problema)","(facile, capire)","(difficile, capire)","(capire, accento)",
                                        "(noleggio, persone)","(capire, inglese)","(rappresentare, parlare)","(difficoltà, capire)",
                                        "(capire, rappresentante)","(capire, godaddy)","(tempo, capire)","(capire, dire)",
                                        "(gentiluomo, parla)"
                                       ] ,
                   
                   "Product": [#English
                               "(cancel, reservation)","(never, use)","(owner, cancel)","(book, property)","(use, service)",
                               "(property, available)","(wait, owner)","(godaddy, system)","(try, book)","(multiple, time)",
                               "(make, reservation)","(contact, owner)","(tell, godaddy)","(book, another)","(complete, book)",
                               "(never, book)","(godaddy, could)","(service, fee)","(rental, company)","(property, manager)",
                               "(home, owner)","(property, owner)","(owner, get)","(need, change)","(request, book)",
                               "(book, godaddy)","(place, stay)","(double, book)","(contact, host)","(use, godaddy)","(godaddy, website)",
                               "(owner, say)","(vacation, rentals)","(web, site)","(get, reservation)","(host, cancel)",
                               "(owner, godaddy)","(able, book)","(email, address)","(book, place)","(email, owner)",
                               "(reservation, cancel)","(property, list)","(godaddy, property)","(rental, agreement)",
                               "(godaddy, service)","(godaddy, site)","(list, property)","(godaddy, future)","(owner, property)",
                               "(godaddy, cancel)","(godaddy, give)","(owner, respond)","(property, owners)","(godaddy, fee)",
                               "(godaddy, say)","(could, book)","(back, forth)","(godaddy, account)","(try, cancel)","(cancel, book)",
                               "(cancel, trip)","(godaddy, owner)","(property, book)","(godaddy, allow)","(need, cancel)",
                               "(need, know)","(cancellation, policy)","(godaddy, take)","(time, book)","(godaddy, book)",
                               "(could, change)","(want, cancel)","(business, godaddy)","(godaddy, app)","(rent, property)",
                               "(rent, godaddy)","(reservation, make)","(travel, insurance)","(godaddy, tell)","(could, cancel)",
                               "(godaddy, issue)","(godaddy, policy)","(book, house)","(change, date)","(issue, property)",
                               "(experience, godaddy)","(list, godaddy)","(confirm, reservation)","(book, request)","(premier, host)",
                               "(reservation, request)","(cancel, due)","(ask, cancel)","(premier, partner)","(godaddy, list)",
                               "(godaddy, get)","(godaddy, platform)","(back, owner)","(godaddy, need)","(policy, godaddy)",
                               "(instant, book)","(cancel, godaddy)","(property, management)","(owner, issue)","(clean, fee)",
                               "(godaddy, charge)","(disappoint, godaddy)","(vacation, rental)","(property, godaddy)","(godaddy, send)",
                               "(property, rent)","(reach, owner)","(contact, property)","(owner, contact)","(deal, godaddy)",
                               "(email, property)","(book, reservation)","(dissatisfy, godaddy)","(love, godaddy)","(get, confirmation)",
                               "(change, email)","(error, message)","(avatar, image)",
                               "(pool, heater)",

                   #Spanish
                       "(cancelar, reserva)","(nunca, usar)","(propietario, cancelar)","(reservar, propiedad)","(usar, servicio)",
                               "(propiedad, disponible)","(esperar, dueño)","(godaddy, sistema)","(probar, reservar)","(múltiples, tiempo)",
                               "(hacer, reserva)","(contacto, propietario)","(decir, padrino)","(libro, otro)","(completar, reservar)",
                               "(nunca, libro)","(godaddy, podría)","(servicio, tarifa)","(alquiler, empresa)","(propiedad, administrador)",
                               "(casa, propietario)","(propiedad, propietario)","(propietario, obtener)","(necesitar, cambiar)","(solicitar, reservar)",
                               "(libro, papi)","(lugar, estancia)","(doble, libro)","(contacto, host)","(uso, papi)","(papá, sitio web)",
                               "(propietario, digamos)","(vacaciones, alquileres)","(web, sitio)","(obtener, reserva)","(anfitrión, cancelar)",
                               "(propietario, padrino)","(capaz, libro)","(correo electrónico, dirección)","(libro, lugar)","(correo electrónico, propietario)",
                               "(reserva, cancelar)","(propiedad, lista)","(godaddy, propiedad)","(alquiler, contrato)",
                               "(godaddy, servicio)","(godaddy, sitio)","(lista, propiedad)","(godaddy, futuro)","(propietario, propiedad)",
                               "(godaddy, cancelar)","(godaddy, dar)","(propietario, responder)","(propiedad, propietarios)","(godaddy, tarifa)",
                               "(godaddy, say)","(could, book)","(adelante, adelante)","(godaddy, cuenta)","(intentar, cancelar)","(cancelar, reservar)",
                               "(cancelar, viaje)","(godaddy, propietario)","(propiedad, libro)","(godaddy, permitir)","(necesitar, cancelar)",
                               "(necesito, saber)","(cancelación, política)","(godaddy, toma)","(tiempo, libro)","(godaddy, libro)",
                               "(podría, cambiar)","(quiero, cancelar)","(negocio, padrino)","(padrino, aplicación)","(alquiler, propiedad)",
                               "(alquilar, padrino)","(reserva, hacer)","(viaje, seguro)","(padrino, contar)","(podria, cancelar)",
                               "(godaddy, problema)","(godaddy, política)","(libro, casa)","(cambio, fecha)","(problema, propiedad)",
                               "(experiencia, padrino)","(lista, padrino)","(confirmar, reserva)","(reservar, solicitar)","(premier, anfitrión)",
                               "(reserva, solicitud)","(cancelar, vencer)","(pedir, cancelar)","(premier, socio)","(godaddy, lista)",
                               "(godaddy, get)","(godaddy, plataforma)","(atrás, propietario)","(godaddy, necesidad)","(política, godaddy)",
                               "(instantánea, reservar)","(cancelar, padrino)","(propiedad, administración)","(propietario, emisión)","(limpiar, tarifa)",
                               "(godaddy, cobrar)","(decepcionar, godaddy)","(vacaciones, alquilar)","(propiedad, godaddy)","(godaddy, enviar)",
                               "(propiedad, alquiler)","(alcance, propietario)","(contacto, propiedad)","(propietario, contacto)","(trato, padrino)",
                               "(correo electrónico, propiedad)","(libro, reserva)","(insatisfecho, padrino)","(amor, padrino)","(obtener, confirmación)",
                               "(cambio, correo electrónico)","(error, mensaje)","(avatar, imagen)",
                               "(piscina, calentador)",
                   #Italian
                       "(cancella, prenotazione)","(mai, usa)","(proprietario, cancella)","(libro, proprietà)","(uso, servizio)",
                               "(proprietà, disponibile)","(aspetta, proprietario)","(godaddy, sistema)","(prova, prenota)","(multiplo, tempo)",
                               "(make, booking)","(contact, owner)","(tell, godaddy)","(book, another)","(complete, book)",
                               "(mai, prenota)","(godaddy, potrebbe)","(servizio, tariffa)","(affitto, azienda)","(proprietà, gestore)",
                               "(casa, proprietario)","(proprietà, proprietario)","(proprietario, ottenere)","(necessità, modifica)","(richiesta, libro)",
                               "(libro, godaddy)","(posto, soggiorno)","(doppio, libro)","(contatto, host)","(uso, godaddy)","(godaddy, sito web)",
                               "(proprietario, diciamo)","(vacanze, affitti)","(web, sito)","(ottenere, prenotazione)","(host, annullare)",
                               "(proprietario, godaddy)","(abili, libro)","(e-mail, indirizzo)","(libro, luogo)","(e-mail, proprietario)",
                               "(prenotazione, annullamento)","(proprietà, elenco)","(godaddy, proprietà)","(affitto, accordo)",
                               "(godaddy, servizio)","(godaddy, sito)","(elenco, proprietà)","(godaddy, futuro)","(proprietario, proprietà)",
                               "(godaddy, cancel)","(godaddy, give)","(proprietario, rispondi)","(proprietà, proprietari)","(godaddy, compenso)",
                               "(godaddy, say)","(could, book)","(avanti, indietro)","(godaddy, account)","(try, cancel)","(cancel, book)",
                               "(cancella, viaggio)","(godaddy, proprietario)","(proprietà, libro)","(godaddy, consenti)","(bisogno, annulla)",
                               "(bisogno, sapere)","(cancellazione, politica)","(godaddy, prendi)","(tempo, libro)","(godaddy, libro)",
                               "(potrebbe, cambiare)","(voglia, annullare)","(affari, godaddy)","(godaddy, app)","(affitto, proprietà)",
                               "(affitto, godaddy)","(prenotazione, fare)","(viaggio, assicurazione)","(godaddy, dillo)","(potrebbe, annullare)",
                               "(godaddy, problema)","(godaddy, politica)","(libro, casa)","(cambiamento, data)","(numero, proprietà)",
                               "(esperienza, godaddy)","(list, godaddy)","(conferma, prenotazione)","(libro, richiesta)","(premier, host)",
                               "(prenotazione, richiesta)","(cancella, dovuto)","(ask, cancel)","(premier, partner)","(godaddy, list)",
                               "(godaddy, get)","(godaddy, platform)","(back, owner)","(godaddy, need)","(policy, godaddy)",
                               "(instant, book)","(cancel, godaddy)","(proprietà, gestione)","(proprietario, problema)","(clean, fee)",
                               "(godaddy, addebito)","(godaddy, godaddy)","(vacanza, affitto)","(proprietà, godaddy)","(godaddy, invia)",
                               "(proprietà, affitto)","(portata, proprietario)","(contatto, proprietà)","(proprietario, contatto)","(affare, godaddy)",
                               "(email, proprietà)","(libro, prenotazione)","(insoddisfatto, godaddy)","(amore, godaddy)","(ricevi, conferma)",
                               "(cambia, email)","(errore, messaggio)","(avatar, immagine)",
                               "(piscina, riscaldamento)",
                   #French
                       "(annuler, réservation)","(jamais, utiliser)","(propriétaire, annuler)","(réserver, propriété)","(utiliser, service)",
                               "(propriété, disponible)","(attendre, propriétaire)","(parrain, système)","(essayer, réserver)","(multiple, temps)",
                               "(faire, réservation)","(contacter, propriétaire)","(dire, godaddy)","(réserver, un autre)","(compléter, réserver)",
                               "(jamais, réserver)","(parrain, pourrait)","(service, frais)","(location, société)","(propriété, gestionnaire)",
                               "(maison, propriétaire)","(propriété, propriétaire)","(propriétaire, obtenir)","(besoin, changement)","(demande, réservation)",
                               "(book, godaddy)","(place, stay)","(double, book)","(contact, host)","(use, godaddy)","(godaddy, website)",
                               "(propriétaire, dites)","(vacances, locations)","(web, site)","(obtenir, réservation)","(hôte, annuler)",
                               "(propriétaire, godaddy)","(capable, réserver)","(email, adresse)","(réserver, lieu)","(email, propriétaire)",
                               "(réservation, annuler)","(propriété, liste)","(parrain, propriété)","(location, contrat)",
                               "(parrain, service)","(parrain, site)","(liste, propriété)","(parrain, futur)","(propriétaire, propriété)",
                               "(parrain, annuler)","(parrain, donner)","(propriétaire, répondre)","(propriété, propriétaires)","(parrain, frais)",
                               "(godaddy, say)","(pourrait, réserver)","(aller, retour)","(godaddy, compte)","(essayer, annuler)","(annuler, réserver)",
                               "(annuler, voyage)","(parrain, propriétaire)","(propriété, réserver)","(parrain, autoriser)","(besoin, annuler)",
                               "(besoin, savoir)","(annulation, politique)","(parrain, prendre)","(heure, livre)","(parrain, livre)",
                               "(pourrait, changer)","(vouloir, annuler)","(business, godaddy)","(godaddy, app)","(loyer, propriété)",
                               "(louer, parrain)","(réservation, faire)","(voyage, assurance)","(parrain, dire)","(pourrait, annuler)",
                               "(parrain, problème)","(parrain, politique)","(livre, maison)","(changement, date)","(problème, propriété)",
                               "(experience, godaddy)","(list, godaddy)","(confirm, reservation)","(book, request)","(premier, host)",
                               "(réservation, demande)","(annuler, échéance)","(demander, annuler)","(premier, partenaire)","(parrain, liste)",
                               "(godaddy, get)","(godaddy, platform)","(back, owner)","(godaddy, need)","(policy, godaddy)",
                               "(instantané, livre)","(annuler, godaddy)","(propriété, gestion)","(propriétaire, problème)","(propre, frais)",
                               "(godaddy, charge)","(décevoir, godaddy)","(vacances, location)","(propriété, godaddy)","(godaddy, envoyer)",
                               "(propriété, location)","(atteindre, propriétaire)","(contact, propriété)","(propriétaire, contact)","(affaire, godaddy)",
                               "(email, propriété)","(book, reservation)","(insatisfy, godaddy)","(love, godaddy)","(get, confirmation)",
                               "(modification, email)","(erreur, message)","(avatar, image)",
                               "(piscine, chauffage)",
                   #German
                       "(stornieren, reservierung)","(niemals verwenden)","(eigentümer, stornieren)","(buch, objekt)","(nutzung, service)",
                               "(Immobilie, verfügbar)","(warten, Besitzer)","(Godaddy, System)","(versuchen, buchen)","(mehrere, Zeit)",
                               "(machen, reservieren)","(kontaktieren, besitzer)","(erzählen, gnädige Frau)","(buchen, noch etwas)","(abschließen, buchen)",
                               "(nie, buchen)","(Godaddy, könnte)","(Service, Gebühr)","(Vermietung, Firma)","(Immobilie, Verwalter)",
                               "(Haus, Eigentümer)","(Grundstück, Eigentümer)","(Eigentümer, Erwerb)","(Bedürfnis, Änderung)","(Anfrage, Buchung)",
                               "(book, godaddy)","(place, stay)","(double, book)","(contact, host)","(use, godaddy)","(godaddy, website)",
                               "(Eigentümer, sagen wir)","(Urlaub, Mietobjekte)","(Web, Website)","(erhalten, reservieren)","(Gastgeber, stornieren)",
                               "(Besitzer, Pate)","(fähig, Buch)","(E-Mail, Adresse)","(Buch, Ort)","(E-Mail, Besitzer)",
                               "(reservierung, stornierung)","(objekt, liste)","(godaddy, objekt)","(miete, vertrag)",
                               "(Godaddy, Service)","(Godaddy, Website)","(Liste, Eigentum)","(Godaddy, Zukunft)","(Eigentümer, Eigentum)",
                               "(Paten, absagen)","(Paten, geben)","(Eigentümer, antworten)","(Eigentum, Eigentümer)","(Paten, Gebühr)",
                               "(Godaddy, sagen)","(könnte, buchen)","(hin, her)","(Godaddy, Konto)","(probieren, stornieren)","(stornieren, buchen)",
                               "(Reise stornieren)","(Paten, Eigentümer)","(Immobilie, Buch)","(Paten, erlauben)","(müssen, stornieren)",
                               "(müssen, wissen)","(stornierung, richtlinie)","(godaddy, take)","(time, book)","(godaddy, book)",
                               "(könnte, ändern)","(wollen, stornieren)","(geschäft, papa)","(papa, app)","(miete, grundstück)",
                               "(miete, papa)","(reservierung, machen)","(reise, versicherung)","(papa, erzähle)","(könnte, stornieren)",
                               "(Paten, Problem)","(Paten, Police)","(Buch, Haus)","(Änderung, Datum)","(Problem, Eigentum)",
                               "(Erfahrung, Pate)","(Liste, Pate)","(Bestätigen, Reservierung)","(Buchen, Anfrage)","(Premier, Gastgeber)",
                               "(reservierung, anfrage)","(stornieren, fällig)","(fragen, stornieren)","(premier, partner)","(godaddy, list)",
                               "(Godaddy, get)","(Godaddy, Plattform)","(Back, Owner)","(Godaddy, Need)","(Policy, Godaddy)",
                               "(Sofort, Buch)","(Stornieren, Pate)","(Immobilie, Verwaltung)","(Eigentümer, Ausgabe)","(Reinigung, Gebühr)",
                               "(Godaddy, Ladung)","(Enttäuschen, Godaddy)","(Urlaub, Miete)","(Eigentum, Godaddy)","(Godaddy, senden)",
                               "(Grundstück, Miete)","(Reichweite, Eigentümer)","(Kontakt, Eigentum)","(Eigentümer, Kontakt)","(Deal, Godaddy)",
                               "(E-Mail, Unterkunft)","(Buch, Reservierung)","(unzufrieden, Pate)","(Liebe, Pate)","(erhalten, Bestätigung)",
                               "(Änderung, E-Mail)","(Fehler, Nachricht)","(Avatar, Bild)",
                               "(Pool, Heizung)",
                   #Turkish
                       "(iptal, rezervasyon)","(asla, kullanma)","(sahip, iptal)","(kitap, mülk)","(kullan, hizmet)",
                               "(mülk, müsait)","(bekle, sahip)","(godaddy, sistem)","(dene, kitap)","(çoklu, zaman)",
                               "(yap, rezervasyon yap)","(irtibat, sahip)","(söyle, godaddy)","(kitap, başka bir)","(tamamla, kitap)",
                               "(asla, kitap)","(godaddy, olabilir)","(hizmet, ücret)","(kiralama, şirket)","(mülk, yönetici)",
                               "(ev, sahip)","(mülk, sahip)","(sahip, al)","(ihtiyaç, değişiklik)","(istek, kitap)",
                               "(kitap, godaddy)","(yer, kal)","(çift, kitap)","(iletişim, ev sahibi)","(kullan, godaddy)","(godaddy, web sitesi)",
                               "(sahip, say)","(tatil, kiralık)","(web, site)","(al, rezervasyon)","(ev sahibi, iptal)",
                               "(sahip, godaddy)","(güçlü, kitap)","(e-posta, adres)","(kitap, yer)","(e-posta, sahip)",
                               "(rezervasyon, iptal)","(mülk, liste)","(godaddy, mülk)","(kiralama, sözleşme)",
                               "(godaddy, hizmet)","(godaddy, site)","(liste, mülk)","(godaddy, gelecek)","(sahip, mülk)",
                               "(godaddy, iptal)","(godaddy, ver)","(sahip, yanıtla)","(mülk, sahipler)","(godaddy, ücret)",
                               "(godaddy, say)","(olabilir, kitap)","(geri, ileri)","(godaddy, hesap)","(dene, iptal et)","(iptal, kitap)",
                               "(iptal, gezi)","(godaddy, sahip)","(mülk, kitap)","(godaddy, izin ver)","(gerekli, iptal)",
                               "(ihtiyaç, bilmek)","(iptal, politika)","(godaddy, al)","(zaman, kitap)","(godaddy, kitap)",
                               "(olabilir, değiştirebilir)","(istebilir, iptal edebilir)","(iş, godaddy)","(godaddy, uygulama)","(kira, mülk)",
                               "(kira, godaddy)","(rezervasyon, yap)","(seyahat, sigorta)","(godaddy, söyle)","(yapabilir, iptal edebilir)",
                               "(godaddy, konu)","(godaddy, politika)","(kitap, ev)","(değişiklik, tarih)","(sayı, mülk)",
                               "(deneyim, godaddy)","(liste, godaddy)","(onayla, rezervasyon)","(kitap, istek)","(premier, ev sahibi)",
                               "(rezervasyon, istek)","(iptal, vadesi geldi)","(iste, iptal et)","(premier, ortak)","(godaddy, liste)",
                               "(godaddy, get)","(godaddy, platform)","(geri, sahip)","(godaddy, ihtiyaç)","(politika, godaddy)",
                               "(anında, kitap)","(iptal, godaddy)","(mülk, yönetim)","(sahip, sorun)","(temiz, ücret)",
                               "(godaddy, ücret)","(hayal kırıklığı, godaddy)","(tatil, kiralık)","(mülk, godaddy)","(godaddy, gönder)",
                               "(mülk, kira)","(erişim, sahip)","(irtibat, mülk)","(sahip, ilgili kişi)","(anlaşma, godaddy)",
                               "(e-posta, mülk)","(kitap, rezervasyon)","(memnuniyetsizlik, godaddy)","(aşk, godaddy)","(al, onay)",
                               "(değişiklik, e-posta)","(hata, mesaj)","(avatar, resim)",
                               "(havuz, ısıtıcı)"
                   ]           
            } 

################### FUNCION PARA CATEGORIZAR ############################################
def get_category(x):
    key_list = []
    for key in Categories_list.keys():
        for value in Categories_list[key]:
            if value in x:
                if key not in key_list:
                    key_list.append(key)
    return(key_list)

################### FUNCION PARA RETORNAR EL BIGRAMA ##################################3

def get_bigram_key(x):
    key_list = []
    for value in Categories_list[x["Category"]]:
        if value in x["bigrams"]:
            if value not in key_list:
                key_list.append(value)
    return(key_list)

# Se aplica la funcion al df:
df["Category"]=df["bigrams"].apply(lambda x: get_category(x))
df["Category"]=df["Category"].map(str)
#df=df.drop("bigrams", axis=1)
df=df[df["Category"]!="[]"]
df["Category"]=df["Category"].map(str).apply(lambda x: x.replace("[",""))
df["Category"]=df["Category"].map(str).apply(lambda x: x.replace("]",""))
df["Category"]=df["Category"].map(str).apply(lambda x: x.replace("'",""))
df["Category"]=df["Category"].map(str).apply(lambda x: x.strip())
df["Category"]=df["Category"].map(str)
df.Category = df.Category.str.split(",")
df = df.explode('Category')
df["Category"]=df["Category"].map(str).apply(lambda x: x.strip())
#Obtener bigramas por categoria: 
df["Category2"]=df.apply(get_bigram_key, axis=1)
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace("[",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace("]",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace("'",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace("), (","-"))
df["Category2"]=df["Category2"].map(str)
df.Category2 = df.Category2.str.split("-")
df = df.explode('Category2')
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace("(",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace(")",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.replace(",",""))
df["Category2"]=df["Category2"].map(str).apply(lambda x: x.strip())
df=df.drop("bigrams", axis=1)
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,...,NpsScore,NPS,Feedback,SiteCode,Jomax,len,Month,Language,Category,Category2
Year,,,,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,...,10,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June,English,Issue Resolution,help issue
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,...,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English,Issue Resolution,take time
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,...,10,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English,Language Skills,understand issue
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,...,10,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June,English,Customer Service,super helpful
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,...,10,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June,English,Customer Service,helpful thank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,0,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Call Issues,phone call
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,0,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Call Issues,hold long
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,0,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Language Skills,hard time


In [71]:
pol= lambda x: TextBlob(x).sentiment.polarity
df["Polarity"]=df['Feedback'].apply(pol)
df

,Date,Time,ShopperId,CustomerType,Guide,Supervisor,ShopperRegionI,SurveyId,Orecxid,CaseProductGroup,...,NPS,Feedback,SiteCode,Jomax,len,Month,Language,Category,Category2,Polarity
Year,,,,,,,,,,,,,,,,,,,,,
2023,2023-06-11,04:56:19,456716782,NaN,Estefani Huaman,"Estrada, Jannet",International - RoW,201592502,NaN,domains,...,Promoter,she's very smart and help me out with the issu...,Perú - Lima,estefani5632,68,June,English,Issue Resolution,help issue,0.278571
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,...,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English,Issue Resolution,take time,0.283333
2023,2023-06-11,03:07:35,81164049,NaN,Bryan Malpartida,"Medina, Andrea",International - RoW,201592506,NaN,account,...,Promoter,bryan is very patient with customers even thou...,Perú - Lima,bryan6953,287,June,English,Language Skills,understand issue,0.283333
2023,2023-06-11,05:38:27,67910895,NaN,Karim Mohamed,"Ashraf, Youstina",International - RoW,201592534,107518899.0,domains,...,Promoter,karim was super helpful and very professional,Egipt - Cairo,karim5737,45,June,English,Customer Service,super helpful,0.231667
2023,2023-06-11,06:24:59,137315074,NaN,Ahmed Masoud,"Ashraf, Youstina",International - RoW,201592537,107519148.0,domains,...,Promoter,professional patient and very helpful thank yo...,Egipt - Cairo,ahmed8164,81,June,English,Customer Service,helpful thank,0.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Call Issues,phone call,-0.015741
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Call Issues,hold long,-0.015741
2022,2022-10-23,08:45:44,300040169,International Independent,Housam ElNassag,"Ashraf, Youstina",International - RoW,201190765,98367094.0,NaN,...,Detractor,he was not professional he wasted my team and...,Egipt - Cairo,housam8378,265,October,English,Language Skills,hard time,-0.015741


In [72]:
df.to_csv("../Data Test/Categorization.csv",encoding="utf-8-sig")

In [73]:
#Tratamiento para Idiomas
df=pd.read_csv("../Raw Datas/LanguageRaw.csv")

def get_language(text):
    try:
        language = detect(text)
    except:
        language = None
    return language

df["Language"] = df["Feedback"].apply(lambda x: get_language(x))
df=df.dropna(subset=["Language"])
df=df[['SurveyId', 'Language']]

In [74]:
df2=pd.read_csv("../Raw Datas/LanDict.csv")
df2=df2.set_index("Language")

In [75]:
df=df.merge(df2, on="Language",how="left")
#df=df[['SurveyId', 'Language','ISO language name_x']]
df.to_csv("../Raw Datas/SurvLang.csv")
df

,SurveyId,Language,ISO language name,639-2/T,639-2/B,639-3,Notes
0,201592493,en,English,eng,eng,eng,NaN
1,201592502,en,English,eng,eng,eng,NaN
2,201592506,en,English,eng,eng,eng,NaN
3,201592534,en,English,eng,eng,eng,NaN
4,201592537,en,English,eng,eng,eng,NaN
...,...,...,...,...,...,...,...
9651,201130921,it,Italian,ita,ita,ita,NaN
9652,201130380,en,English,eng,eng,eng,NaN
9653,201130386,en,English,eng,eng,eng,NaN
9654,201130346,tr,Turkish,tur,tur,tur,NaN


In [76]:
print("done", dt.now())

done 2023-06-13 11:57:13.079311


In [77]:
from datetime import datetime, timedelta, date
import time


start_date = date(2023, 5, 24) 
end_date = date(2023, 5, 29)    # perhaps date.now()

delta = end_date - start_date
for i in range(delta.days + 1):
    date = start_date + timedelta(days=i)
    date =datetime.strftime(date, '%m/%d/%Y')
    print(date)

05/24/2023
05/25/2023
05/26/2023
05/27/2023
05/28/2023
05/29/2023
